# AimBig Star Ratings Prediction 

### Extracting the Data - for __________ Star Rating (1)
- From 1st Star Rating assessment period (for the Sept 2022 Star Rating)
- Assessment period: 1/10/20-7/10/22 (8 quarters)

In [460]:
#Packages:
import datetime as dt
import re
import json
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from datetime import date, timedelta
from datetime import datetime
import warnings
import math
import seaborn as sn
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')


### Assessment Periods - CHANGE HERE

In [461]:
endDate = pd.to_datetime('2021-09-30', format='%Y-%m-%d')
startDate = pd.to_datetime('2019-10-01', format='%Y-%m-%d')

# Assessment period month and year -> for string naming 
period = 'september_2021'

- Need to split DMS, ESS -> to apply performance weightings

### To-Do 20/09 
- Subset each of the dataframes to include attributes needed for star rating calculation - done
- Join dataframes to do with regression analysis -> compare attribute values with the "JSCI Personal Factors Outcome"
- Get the performance measure (current star ratings report) - and filter for the data in the 1st star rating period - done
- Compare star rating calculation with filtered data. - in progress

### Analysing from Participant -> Site -> Contract Level 


## Step 1) Actual Performance vs. Expected Performance 
Performance: numerator/denominator for caseload per contract
- Consider DMS and ESS contracts separately
- Only focusing on full outcomes (13, 26, 52 week full outcomes)

- Calculating the performance (actual vs expected) every quarter 

Placement_status: if participant has commenced employment
Placement_Phase: PPS: post-placement-support

Quarterisation:
- Look at example from government pdf (if a person doesn't achieve) - Figure 1
- Check which records is included for a participant 
- Pre-quarterisation: Looking at a participants overall period with a contract (from start date -> end date)
    - This period doesn't consider the start date to end date (broken up by quarters)
- Post-quarterisation: Breaking up the pre-quarterisation period (from start date - to end date) by quarters. Looking at whether a participant has achieved an outcome in each quarter (within the start-end date). If they haven't, then the denominator = 1, and overall denominator += 1. 


## TO-DO 27/09-28/09:
1. Pre-quarterisation periods - start pre-quarterisation period analysis.
    1.1 Overview of participants with a contract (start date -> end date) - done
    
    1.2 LifeCycle File: Investigate the job_seeker_id, placement_status_code, placement_phase_code, placement_sequence_number. The dates: Referral_Date, Commencement_Date, Time_In_Program_Weeks - done

    1.3 From 1.1 - Investigate the participants who numerate/denominate. - Using claims file (after joining to the two other files)
        - Split data into 13, 26, 52 FULL outcomes (Performance measures)
        - Records not included in numerator or denominator: record is not included, as the participant time to achieve an outcome is less than 13/26/52 weeks (check commencement date/outcome evaluation date)

    1.4 Compare pre-quarterisation period with expected performance (calculated by Arriba Group)
    
    1.5 Join job_seeker_id to caseload file. - done
    


- After each quarter, the placement status code is recorded. It is sustained if there are no changes to the status code since the previous quarter. 


Working: 
- Need to be still in assistance at the end of the two year rolling period.
COM_EA: commenced employment with assistance
COM_PPS: commenced employment with post placement support 
COM_OS: commenced employment with ongoing support 
TRN_PPS: transferred with post placement support 
TRN_EA: transferred with employment assistance



- Include these outcomes - given the date period

Exited:
- Exited the site during the two year rolling period
- Include both of these:
EXT_EA: exited the site with assistance, or exited assistance
EXT_PPS: exited the site with post placement support 

exited the Site prior to the two year rolling period but have an associated Outcome which was lodged during the two year period at the Site; or
- Get this from claims 
- Note: this won't be from lifecycle file

exited prior to the two year rolling period but had an ongoing support employment verification date which is within the two year rolling period.
Variables: OSA Date, OSA Due Date (needs to be in the 2 year rolling period)
- don't denominate or numerate

SUS_EA: suspended with employment assistance (Counted in denominator if completed more than 13 weeks with us. Includes suspension period)


### Pathway Meaning
- Include a table with numerator/denominator boolean columns -> initial plan
- Determining whether a combination should be included in a numerator/denominator or both

### Contract Level - 13, 26, 52 outcomes: participants performance on a contract level

## Claims and Recording Outcomes
- The COMMENCEMENT_DATE column: marks the date from which a participant is expected to record an outcome. 
- Numerates: 13/26/52 weeks from COMMENCEMENT_DATE - an outcome is achieved.
- COMMENCEMENT_DATE changes based on the PLACEMENT_SEQUENCE_NUMBER.
- Participant needs to numerate a 13wk full outcome before numerating a 26wk full outcome.
- OUTCOME_TYPE: employment only
- Don't count the service fee.
- Edge case: If no claim is made 13wks after the COMMENCEMENT_DATE, but a 26wk claim is made after 13wk - sometimes 13wk claim is made on the same date (late).
- Service fee claims are not counted in numerator
- We get outcome claims when participant is in the PPS phase.
- Once a participant completes 26 weeks - they denominate in 26 weeks, etc.
- Calculating num/denom separately for each full outcome type


In [462]:
## Loading the comb df in:
file_string = 'C:/Users/AManalo/OneDrive - Arriba Group/Desktop/Star Ratings - AimBig/STR Prediction Model/Data Files/' + period + '/apv_stream_participants_' + period + '.csv'
df_comb =  pd.read_csv(file_string) # reading from github repo

df_comb = df_comb.rename(columns={'Exit_Date' : 'EXIT_DATE', 'placement_type' : 'PLACEMENT_TYPE'})




In [463]:
# drop rows where COMMENCEMENT_DATE_NEW is NULL -> no valid placements
df_comb = df_comb[df_comb['COMMENCEMENT_DATE'].isna() == False]
# drop None
df_comb = df_comb[df_comb['COMMENCEMENT_DATE'] != None]

In [464]:
df_comb.columns

Index(['COMMENCEMENT_DATE', 'Program_Type_Code', 'jr_jobseeker_id',
       'Job_Seeker_ID', 'plac_jobseeker_ID', 'PLACEMENT_TYPE', 'Program',
       'Contract_ID', 'Site_Name', 'ESA_Code', 'specialist_site_type_code',
       'Referral_Date', 'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE',
       'Phase_changed', 'Status_changed', 'Phase_commencement_date',
       'Time_In_Program_Weeks', 'Period_of_Service_Weeks', 'Disability_Type',
       'moderate_intellectual_disability', 'indigenous', 'volunteer',
       'jsci_status', 'employment_benchmark', 'esat_jca_assessment_date',
       'EXIT_DATE', 'SUSPENSION_FROM_DATE', 'SUSPENSION_ACTUAL_END_DATE',
       'CLAIM_AVAILABLE_DATE', 'Claim_Rate_Type', 'Claim_Status', 'Claim_ID',
       'Claim_Type', 'OUTCOME_TYPE', 'CLAIM_START_DATE', 'CLAIM_END_DATE',
       'overridden_outcome_started_on', 'pps_start_date'],
      dtype='object')

In [465]:
df_comb

,COMMENCEMENT_DATE,Program_Type_Code,jr_jobseeker_id,Job_Seeker_ID,plac_jobseeker_ID,PLACEMENT_TYPE,Program,Contract_ID,Site_Name,ESA_Code,...,CLAIM_AVAILABLE_DATE,Claim_Rate_Type,Claim_Status,Claim_ID,Claim_Type,OUTCOME_TYPE,CLAIM_START_DATE,CLAIM_END_DATE,overridden_outcome_started_on,pps_start_date
1,2019-11-22 00:00:00,DMS,2343,5114405109,2343,Vacancy Placement,DMS,0212820K,AimBig Employment Pty Ltd RESERVOIR,4PLE,...,NaN,NaN,NaN,7754,Des week 4,NaN,NaN,NaN,NaN,2019-11-22
2,2019-11-22 00:00:00,DMS,2343,5114405109,2343,Vacancy Placement,DMS,0212820K,AimBig Employment Pty Ltd RESERVOIR,4PLE,...,NaN,NaN,NaN,7795,Des week 52,NaN,NaN,NaN,NaN,2019-11-22
3,2019-11-22 00:00:00,DMS,2343,5114405109,2343,Vacancy Placement,DMS,0212820K,AimBig Employment Pty Ltd RESERVOIR,4PLE,...,NaN,NaN,NaN,7786,Des week 26,NaN,NaN,NaN,NaN,2019-11-22
5,2019-11-22 00:00:00,DMS,2343,5114405109,2343,Vacancy Placement,DMS,0212820K,AimBig Employment Pty Ltd RESERVOIR,4PLE,...,NaN,NaN,NaN,7775,Des week 13,NaN,NaN,NaN,NaN,2019-11-22
12,2021-07-29 00:00:00,DMS,2941,2738240909,2941,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,...,NaN,NaN,NaN,63058,Des week 52,NaN,NaN,NaN,NaN,2021-07-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295737,2021-04-28 00:00:00,DMS,6461,3521776609,6461,Vacancy Placement,DMS,0212796E,AimBig Employment WOOLLOONGABBA,4SOB,...,NaN,NaN,NaN,54378,Des week 52,NaN,NaN,NaN,NaN,2021-04-28
295738,2021-07-05 00:00:00,DMS,4380,6676602609,4380,Vacancy Placement,DMS,0212820K,AimBig Employment EPPING,4PLE,...,NaN,NaN,NaN,60973,Des week 52,NaN,NaN,NaN,NaN,2021-07-05
295739,2021-07-05 00:00:00,DMS,4380,6676602609,4380,Vacancy Placement,DMS,0212820K,AimBig Employment EPPING,4PLE,...,NaN,NaN,NaN,60971,Des week 13,NaN,NaN,NaN,NaN,2021-07-05
295740,2021-07-05 00:00:00,DMS,4380,6676602609,4380,Vacancy Placement,DMS,0212820K,AimBig Employment EPPING,4PLE,...,NaN,NaN,NaN,60970,Des week 4,NaN,NaN,NaN,NaN,2021-07-05


In [466]:
len(df_comb)

212132

In [467]:
# Assigning it to df_pre
df_pre = df_comb[["Job_Seeker_ID", 'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE', 'PLACEMENT_TYPE', 'Program',
       'Contract_ID', 'Site_Name', 'ESA_Code',  'specialist_site_type_code', 'COMMENCEMENT_DATE', 'EXIT_DATE', 'Claim_ID', 'Claim_Rate_Type', 'Claim_Status', 'CLAIM_START_DATE', 'CLAIM_AVAILABLE_DATE', 'CLAIM_END_DATE', 'SUSPENSION_FROM_DATE', 'SUSPENSION_ACTUAL_END_DATE', 'OUTCOME_TYPE']]
df_pre = df_pre.sort_values(by=['Job_Seeker_ID', 'CLAIM_START_DATE'])

# converting data types + converting NaT to out of range dates (i.e. 2001)
df_pre['Job_Seeker_ID'] = pd.to_numeric(df_pre['Job_Seeker_ID'], errors='coerce')
df_pre = df_pre[df_pre['Job_Seeker_ID'].isna() != True]
df_pre['Job_Seeker_ID'] = df_pre['Job_Seeker_ID'].astype('int64')


# need to convert column into string first -> then back to date
dict_types = {'SUSPENSION_FROM_DATE' : 'str', 'COMMENCEMENT_DATE' : 'str', 'CLAIM_START_DATE' : 'str', 'CLAIM_AVAILABLE_DATE' : 'str', 'CLAIM_END_DATE' : 'str', 'SUSPENSION_ACTUAL_END_DATE' : 'str' }
df_pre = df_pre.astype(dict_types)


# fill NA values with out of bounds date
fillVal = pd.to_datetime('1/01/2001', format='%d/%m/%Y')




df_pre['SUSPENSION_FROM_DATE'] = pd.to_datetime(df_pre['SUSPENSION_FROM_DATE'], format='%Y-%m-%d', exact=False,errors='coerce')
df_pre['SUSPENSION_FROM_DATE'] = df_pre['SUSPENSION_FROM_DATE'].fillna(fillVal)


df_pre['SUSPENSION_ACTUAL_END_DATE'] = pd.to_datetime(df_pre['SUSPENSION_ACTUAL_END_DATE'], format='%Y-%m-%d', exact=False, errors='coerce')
df_pre['SUSPENSION_ACTUAL_END_DATE'] = df_pre['SUSPENSION_ACTUAL_END_DATE'].fillna(fillVal)

df_pre['COMMENCEMENT_DATE'] = pd.to_datetime(df_pre['COMMENCEMENT_DATE'], format='%Y-%m-%d', exact=False, errors='coerce')
df_pre['COMMENCEMENT_DATE'] = df_pre['COMMENCEMENT_DATE'].fillna(fillVal)


df_pre['CLAIM_START_DATE'] = pd.to_datetime(df_pre['CLAIM_START_DATE'], format='%Y-%m-%d', exact=False, errors='coerce')
df_pre['CLAIM_START_DATE'] = df_pre['CLAIM_START_DATE'].fillna(fillVal)

df_pre['CLAIM_AVAILABLE_DATE'] = pd.to_datetime(df_pre['CLAIM_AVAILABLE_DATE'], format='%Y-%m-%d', exact=False, errors='coerce')
df_pre['CLAIM_AVAILABLE_DATE'] = df_pre['CLAIM_AVAILABLE_DATE'].fillna(fillVal)

df_pre['CLAIM_END_DATE'] = pd.to_datetime(df_pre['CLAIM_END_DATE'], format='%Y-%m-%d', exact=False, errors='coerce')
df_pre['CLAIM_END_DATE'] = df_pre['CLAIM_END_DATE'].fillna(fillVal)



df_pre["Claim_Rate_Type"] = df_pre["Claim_Rate_Type"].replace(np.nan, 'na', regex=True)
df_pre["Claim_Rate_Type"] = df_pre["Claim_Rate_Type"].replace('', 'na', regex=True)
df_pre["Claim_Rate_Type"] = df_pre["Claim_Rate_Type"].astype('str')



# remove duplicates - make copy of duplicate free dataframe 
df_pre = df_pre.loc[:,~df_pre.columns.duplicated()].copy()

### Filling in the 'specialist_site_type_code' blanks - THIS HAS BEEN DONE IN EXCEL MANUALLY
- Filled in using the existing codes with corresponding contract_id and site_description
- First groupBy the object
- Then iterate and fillNa values 

How to access the specialist_site_type_code using the (Site_Name, Contract_ID):

### Performance - Numerator
- Aggregate numerator per job_seeker_id
- aggregate numerator for each type of full outcome
- Decide what to do for denominators..

## Denominators
- Duration at assistance is at least 13/26/52 weeks (excluding suspension periods)
- Excludes participants who have achieved a pathway outcome

In [468]:
# checking suspensions or early exits -> adding 13, 26, 52 weeks to the commencement date
week_13 = dt.timedelta(days = 91)
week_26 = dt.timedelta(days = 182)
week_52 = dt.timedelta(weeks = 364)


# getting the period between suspension_from_date and suspension_actual_end_date in weeks
df_pre['SUSPENSION_DAYS'] = (df_pre['SUSPENSION_ACTUAL_END_DATE'] - df_pre['SUSPENSION_FROM_DATE']).apply(lambda x: x/np.timedelta64(1, 'D'))

df_pre['SUSPENSION_DAYS'] = df_pre['SUSPENSION_DAYS'].astype('int64')

df_pre.head(10)



,Job_Seeker_ID,REFERRAL_PHASE_CODE,PLACEMENT_STATUS_CODE,PLACEMENT_TYPE,Program,Contract_ID,Site_Name,ESA_Code,specialist_site_type_code,COMMENCEMENT_DATE,...,Claim_ID,Claim_Rate_Type,Claim_Status,CLAIM_START_DATE,CLAIM_AVAILABLE_DATE,CLAIM_END_DATE,SUSPENSION_FROM_DATE,SUSPENSION_ACTUAL_END_DATE,OUTCOME_TYPE,SUSPENSION_DAYS
57631,61780,PPS,Commenced,Vacancy Placement,ESS,0212898K,AimBig Employment PARRAMATTA,4CWS,NaN,2019-09-30,...,14553,na,NaN,2001-01-01,2001-01-01,2001-01-01,2001-01-01,2001-01-01,NaN,0
57632,61780,PPS,Commenced,Vacancy Placement,ESS,0212898K,AimBig Employment PARRAMATTA,4CWS,NaN,2019-09-30,...,14551,na,NaN,2001-01-01,2001-01-01,2001-01-01,2001-01-01,2001-01-01,NaN,0
57633,61780,PPS,Commenced,Vacancy Placement,ESS,0212898K,AimBig Employment PARRAMATTA,4CWS,NaN,2019-09-30,...,14554,na,NaN,2001-01-01,2001-01-01,2001-01-01,2001-01-01,2001-01-01,NaN,0
57683,61780,PPS,Commenced,Vacancy Placement,ESS,0212898K,AimBig Employment PARRAMATTA,4CWS,NaN,2019-09-30,...,14552,na,NaN,2001-01-01,2001-01-01,2001-01-01,2001-01-01,2001-01-01,NaN,0
97528,61780,EA,Commenced,Vacancy Placement,ESS,0212898K,AimBig Employment PARRAMATTA,4CWS,NaN,2019-09-30,...,14553,na,NaN,2001-01-01,2001-01-01,2001-01-01,2001-01-01,2001-01-01,NaN,0
97529,61780,EA,Commenced,Vacancy Placement,ESS,0212898K,AimBig Employment PARRAMATTA,4CWS,NaN,2019-09-30,...,14551,na,NaN,2001-01-01,2001-01-01,2001-01-01,2001-01-01,2001-01-01,NaN,0
97589,61780,EA,Commenced,Vacancy Placement,ESS,0212898K,AimBig Employment PARRAMATTA,4CWS,NaN,2019-09-30,...,14554,na,NaN,2001-01-01,2001-01-01,2001-01-01,2001-01-01,2001-01-01,NaN,0
97590,61780,EA,Commenced,Vacancy Placement,ESS,0212898K,AimBig Employment PARRAMATTA,4CWS,NaN,2019-09-30,...,14552,na,NaN,2001-01-01,2001-01-01,2001-01-01,2001-01-01,2001-01-01,NaN,0
176747,61780,EA,Transferred,Vacancy Placement,ESS,0212898K,AimBig Employment PARRAMATTA,4CWS,NaN,2019-09-30,...,14553,na,NaN,2001-01-01,2001-01-01,2001-01-01,2001-01-01,2001-01-01,NaN,0
176748,61780,EA,Transferred,Vacancy Placement,ESS,0212898K,AimBig Employment PARRAMATTA,4CWS,NaN,2019-09-30,...,14551,na,NaN,2001-01-01,2001-01-01,2001-01-01,2001-01-01,2001-01-01,NaN,0


In [469]:
df_pre['REFERRAL_PHASE_CODE'].drop_duplicates()

57631     PPS
97528      EA
91672      OS
31236     NaN
185463     NE
Name: REFERRAL_PHASE_CODE, dtype: object

In [470]:
# for denominator: include working codes 
# since referral phase code can only be PPS or EA
ref_pps = (df_pre["REFERRAL_PHASE_CODE"] == 'PPS')
ref_ea = (df_pre["REFERRAL_PHASE_CODE"] == 'EA')
ref_os = (df_pre["REFERRAL_PHASE_CODE"] == 'OS')
com = (df_pre["PLACEMENT_STATUS_CODE"] == 'Commenced')
trn = (df_pre["PLACEMENT_STATUS_CODE"] == 'Transferred')

com_ea = com & ref_ea
com_pps = com & ref_pps
com_os = com & ref_os 
trn_pps = trn & ref_pps
trn_ea = trn & ref_ea 


# include exit codes - but check date (exited during 2 year rolling period): 
# just do start of 2018
date_range = df_pre['COMMENCEMENT_DATE'].loc[(df_pre['COMMENCEMENT_DATE'] >= startDate) & (df_pre['COMMENCEMENT_DATE'] <= endDate)]


ext = (df_pre["REFERRAL_PHASE_CODE"] == 'Exited')
#ext_ea = (ext & ref_ea) & date_range
#ext_pps = ext & ref_pps & date_range

# exited the site prior to the 2 year rolling period but has an outcome lodged during the 2 year rolling period -> doesn't denominate - but doesn't numerate 
#date_before = df_pre['COMMENCEMENT_DATE'].loc[(df_pre['COMMENCEMENT_DATE'] < '2019-01-01')]

# doesn't denominate or numerate
#clm_date = df_pre['CLAIM_START_DATE'].loc[(df_pre['CLAIM_START_DATE'] >= '2019-01-01') & (df_pre['CLAIM_END_DATE'] <= '2021-01-08')]
#no_inc = date_before & clm_date # don't include in num/den

# ext_before = ext & date_before # don't include in num/den


# will count in denominator if sus_ea + more than 13 week + includes suspension period: 
# SUSPENSION IS BASED ON DATE, NOT CODE. Suspension date can occur in any phase
# 1) Suspension period exists: 
is_sus = (df_pre['SUSPENSION_DAYS'].isna() == False) # suspension dates exist and check commencement period 


df_pre.columns

Index(['Job_Seeker_ID', 'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE',
       'PLACEMENT_TYPE', 'Program', 'Contract_ID', 'Site_Name', 'ESA_Code',
       'specialist_site_type_code', 'COMMENCEMENT_DATE', 'EXIT_DATE',
       'Claim_ID', 'Claim_Rate_Type', 'Claim_Status', 'CLAIM_START_DATE',
       'CLAIM_AVAILABLE_DATE', 'CLAIM_END_DATE', 'SUSPENSION_FROM_DATE',
       'SUSPENSION_ACTUAL_END_DATE', 'OUTCOME_TYPE', 'SUSPENSION_DAYS'],
      dtype='object')

In [471]:
df_pre['Claim_Rate_Type'].value_counts()

na                                                    143871
DES 4wk Outcome                                        30013
DES 13wk Full Outcome                                  16963
DES 26wk Full Outcome                                  11339
DES 52wk Full Outcome                                   2765
DES 13wk Full Outcome Education                         2692
DES 26wk Full Outcome Education                         2133
DES 13wk Pathway Outcome                                1251
DES 13wk Pathway Outcome Education                       538
DES 26wk Pathway Outcome                                 338
DES 52wk Pathway Outcome                                 140
DES 4wk Outcome Moderate Intellectual Disability P        29
DES 13wk Outcome Moderate Intellectual Disability         21
DES 26wk Work Assist Outcome                              16
DES 26wk Outcome Moderate Intellectual Disability          9
DES 13wk Full Outcome Zero Dollar R4                       8
DES 52wk Outcome Moderat

In [472]:
find13 = df_pre["Claim_Rate_Type"].str.contains(r'DES 13wk Full Outcome')
find26 = df_pre["Claim_Rate_Type"].str.contains(r'DES 26wk Full Outcome')
find52 = df_pre["Claim_Rate_Type"].str.contains(r'DES 52wk Full Outcome')



# combine the placement_status_code and referral_phase_code :
df_pre['PLACE_REF'] = list(zip(df_pre.PLACEMENT_STATUS_CODE, df_pre.REFERRAL_PHASE_CODE))




### Filtering NaT for - Claim_available_date

- Validating the date period

In [473]:

df_pre['CLAIM_AVAILABLE_DATE'] = pd.to_datetime(df_pre['CLAIM_AVAILABLE_DATE'])
df_pre['COMMENCEMENT_DATE'].min()

Timestamp('2018-05-04 00:00:00')

In [474]:

df_pre['CLAIM_AVAILABLE_DATE'].max()

Timestamp('2021-09-30 00:00:00')

In [475]:
df_pre.columns

Index(['Job_Seeker_ID', 'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE',
       'PLACEMENT_TYPE', 'Program', 'Contract_ID', 'Site_Name', 'ESA_Code',
       'specialist_site_type_code', 'COMMENCEMENT_DATE', 'EXIT_DATE',
       'Claim_ID', 'Claim_Rate_Type', 'Claim_Status', 'CLAIM_START_DATE',
       'CLAIM_AVAILABLE_DATE', 'CLAIM_END_DATE', 'SUSPENSION_FROM_DATE',
       'SUSPENSION_ACTUAL_END_DATE', 'OUTCOME_TYPE', 'SUSPENSION_DAYS',
       'PLACE_REF'],
      dtype='object')

## 13 Week Outcomes (Numerator + Denominator)
+ Creating a dictionary (key: Job_Seeker_ID, values: [all attributes for one row], [all attributes for second row], etc.) 
    - The values in the dictionary are the rows which correspond to the same Job_Seeker_ID -> but stored like so: 
    E.g. for job_seeker_id 1001:
    1001     :  [row 1]
                [row 2]
                [row 3]
    This allows for efficient access when identifying the smallest placement_seq_no for instance -> accessing row 1, rather than iterating through the dataframe. 
    - Can check index of the rows -> need to check whether if row 3 has a 26 week outcome, check if 13 week outcome is in row 2 or row 1. 
    - Check 13 week outcome by adding 13 weeks to commencement date by adding 13x7 days.

In [476]:
df_pre = df_pre.sort_values(by=['COMMENCEMENT_DATE'])
df_pre.head(10)

,Job_Seeker_ID,REFERRAL_PHASE_CODE,PLACEMENT_STATUS_CODE,PLACEMENT_TYPE,Program,Contract_ID,Site_Name,ESA_Code,specialist_site_type_code,COMMENCEMENT_DATE,...,Claim_Rate_Type,Claim_Status,CLAIM_START_DATE,CLAIM_AVAILABLE_DATE,CLAIM_END_DATE,SUSPENSION_FROM_DATE,SUSPENSION_ACTUAL_END_DATE,OUTCOME_TYPE,SUSPENSION_DAYS,PLACE_REF
214165,9386301003,EA,Suspended,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,NaN,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Suspended, EA)"
216097,9386301003,EA,Commenced,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,NaN,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Commenced, EA)"
216096,9386301003,EA,Commenced,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,NaN,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Commenced, EA)"
214162,9386301003,EA,Suspended,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,NaN,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Suspended, EA)"
214161,9386301003,EA,Suspended,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,NaN,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Suspended, EA)"
190363,9386301003,EA,Exited,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,NaN,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Exited, EA)"
190360,9386301003,EA,Exited,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,NaN,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Exited, EA)"
190359,9386301003,EA,Exited,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,NaN,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Exited, EA)"
175014,9386301003,PPS,Commenced,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,NaN,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Commenced, PPS)"
175011,9386301003,PPS,Commenced,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,NaN,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Commenced, PPS)"


In [477]:
df_pre = df_pre.drop_duplicates()
df_pre = df_pre.sort_values(['COMMENCEMENT_DATE', 'CLAIM_START_DATE'])

g = df_pre.groupby(["Job_Seeker_ID"]) # remove commencement_date with NaT -> check if NaT values, filter these out and remove




- Groups -> one group containing entries that correspond to the same Job_Seeker_ID
srce: https://stackoverflow.com/questions/22691010/how-to-print-a-groupby-object


## 13 Week Numerator and Denominator Function - Pre-Quarterisation
- Get the lowest index value of the group (for each list)
- Create a new dataframe df_13: [Job_Seeker_ID, Num_13, Den_13] 
    - Contains the 13 week numerator and denominator for corresponding job seeker id
- Use pd.DateOffset instead of timedelta(days=....)
- Excludes in the denominator: participants who have achieved a 13 week pathway outcome

In [478]:
# list that contains the numerator codes: 
com_ea = com & ref_ea
com_pps = com & ref_pps
com_os = com & ref_os 
trn_pps = trn & ref_pps
trn_ea = trn & ref_ea 
num_codes = [('Commenced', 'EA'), ('Commenced', 'PPS'), ('Commenced', 'OS'), ('Transferred', 'PPS'), ('Transferred', 'EA')]


#EXT_EA: exited the site with assistance, or exited assistance
#EXT_PPS: exited the site with post placement support 
den_codes = [('Exited', 'EA'), ('Exited', 'PPS'), ('Transferred', 'PPS'), ('Transferred', 'EA')]

# participant has commenced at a site:
com_codes = [('Commenced', 'EA'), ('Commenced', 'PPS')]


Job_Seeker_ID is set as the index -> access by "g.index[0]"

In [479]:
com_codes


[('Commenced', 'EA'), ('Commenced', 'PPS')]

In [480]:
df_pre['PLACEMENT_STATUS_CODE'].value_counts()

Commenced      36769
Exited         17132
Pending        15848
Suspended       7807
Transferred     3217
In Progress      160
Name: PLACEMENT_STATUS_CODE, dtype: int64

In [481]:
com_codes

[('Commenced', 'EA'), ('Commenced', 'PPS')]

In [482]:
df_pre['COMMENCEMENT_DATE'].max()

Timestamp('2022-02-16 00:00:00')

In [483]:
def get_13_outcome(jobSeekerEntry, num_codes, den_codes, com_codes):
    startBounds = startDate
    endBounds = endDate
    jobSeekerEntry = jobSeekerEntry.sort_values(by=['COMMENCEMENT_DATE'])
    group = jobSeekerEntry
    id = group.index.values.astype(int)[0] 
    # initialise numerator and denominator 
    num_13 = 0
    den_13 = 0
    df_13 = pd.DataFrame()
    # place id into dataframe
    df_13['Job_Seeker_ID'] = [group.Job_Seeker_ID.values[0]]
    
    # TEMP ----------- REMOVE
    df_13['Site_Name'] = [group.Site_Name.values[0]]
    df_13['Contract_ID'] = [group.Contract_ID.values[0]]
    df_13['specialist_site_type_code'] = [group.specialist_site_type_code.values[0]]

    # TEMP -------------------
    week_13 = pd.DateOffset(days = 91)
    # Check number of entries in group
    targetStr = 'DES 13wk Full Outcome'
    num_entries = int(len(jobSeekerEntry))

    # CLAIM TYPES - 
    eduStr = ['DES 13wk Full Outcome Education', 'DES 13wk Pathway Outcome Education'] # denominate
    noClaim = 'na'
    pathwayStr = 'DES 13wk Pathway Outcome' # doesn't denominate, doesn't numerate
    valid_claims = ['Approved', 'Pending', 'Lodged'] #but excluding lodged, pending or approved FOR EDUCATION 13 week full outcomes
    exitDate = group.EXIT_DATE.values[0] # if exit date exists -> 

    # now loop ->
    j = 0
    comDate = group.COMMENCEMENT_DATE.values[0]
    exitDate = group.EXIT_DATE.values[0] 



    ### check that COMMENCEMENT_DATE IS NOT '2001-01-01'
    inavlid = pd.to_datetime('2001-01-01', format='%Y-%m-%d')
    if comDate == inavlid:
        df_13['Num_13'] = 0
        df_13['Den_13'] = 0
        return df_13
         
    while j < len(group.COMMENCEMENT_DATE.values):
        # commencement check ->
        if group.PLACEMENT_STATUS_CODE.values[j] == 'Commenced':
            
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_13 = comDate + week_13
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension 
                # check suspension days is in the 2 year assessment period:
                if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_13): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_13 = comDate + week_13 + sus_days
            else:
                pass

            # claim rate types:
            if group.Claim_Rate_Type.values[j] == targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                
                # check if participant exits before claim_available_date ->
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if (exitDate < period_13):
                    
                    num_13 = 0 
                    den_13 = 0
                    
                else:
                    num_13 += 1
                    den_13 = 1
                    df_13['Num_13'] = [num_13]
                    df_13['Den_13'] = den_13
                    return df_13 

            elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_13 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if exitDate < period_13: # duration in assistance is less than 13 weeks
                    num_13 = 0 
                    den_13 = 0
                   
                else: # check duration in assistance is at least period_13 
                    num_13 = 0
                    den_13 = 1 #education outcomes are counted in the denominator
            elif group.Claim_Rate_Type.values[j] == pathwayStr and (group.Claim_Status.values[j] in valid_claims): # denominator excludes pathway outcomes
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if (exitDate < period_13):
                    num_13 = 0 
                    den_13 = 0
                else:
                    num_13 += 0 
                    
                    
                    break
            elif group.Claim_Rate_Type.values[j] == noClaim: 
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            
                if (exitDate < period_13):
                    num_13 = 0 
                    
                else:
                   
                    num_13 += 0 
                    den_13 = 1
                   
            else: # either 26, 52 week claim
                num_13 = 0
                
            
        
        # has not commenced ->
        else: 
            
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_13 = comDate + week_13
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension 
                # check suspension days is in the 2 year assessment period:
                if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_13): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_13 = comDate + week_13 + sus_days

            # check if code is SUS_EA: (Counted in denominator if completed more than 13 weeks with us. Includes suspension period)
            if group.PLACE_REF.values[j] == 'SUS_EA':
                
                if group.CLAIM_AVAILABLE_DATE.values[j] >= period_13:
                    if (group.Claim_Rate_Type.values[j] == targetStr) and (group.Claim_Status.values[j] in valid_claims):
                        if (period_13 <= startBounds) or (period_13 >= endBounds):
                                num_13 = 0 
                                den_13 = 0 # out of bounds
                                
                        else:
                            # then doesn't denominate: 
                            num_13 = 1
                            den_13 = 1
                            

                      
                       
                    else:
                        if (period_13 <= startBounds) or (period_13 >= endBounds):
                            num_13 = 0 
                       
                            
    
                        else:
                            if (group.Claim_Rate_Type.values[j] == 'DES 13wk Pathway Outcome'): #this outcome IS NOt in the denominator
                                den_13 = 0 
                  
                       
                    
                else: # no claim available date:
                    if group.Claim_Rate_Type.values[j] == 'na':
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate > period_13:
                            num_13 = 0
                            den_13 = 1
                            break

                    else:
                        num_13 = 0
                        

            else: # TRN or EXT
                
                ext_tfn = ['Exited', 'Transferred']
              
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                
            

                
                # check claim rate types ->
                if (group.Claim_Rate_Type.values[j] == targetStr) and (group.Claim_Status.values[j] in valid_claims): # valid claim
                
                    if exitDate < group.CLAIM_AVAILABLE_DATE.values[j]:
                        if comDate > exitDate: # if participant commences again after exiting then can record outcome ->
                            num_13 += 1
                            den_13 = 1
                            df_13['Num_13'] = [num_13]
                            df_13['Den_13'] = den_13
                            return df_13  
                        else:
                        
                            num_13 = 0
                           
                    else:
                        num_13 += 1
                        den_13 = 1
                        df_13['Num_13'] = [num_13]
                        df_13['Den_13'] = den_13
                        return df_13  

                elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_13 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                    if exitDate < group.CLAIM_AVAILABLE_DATE.values[j]:
                        if comDate > exitDate: # if participant commences again after exiting then can record outcome ->
                            den_13 = 1
                            df_13['Num_13'] = [num_13]
                            df_13['Den_13'] = den_13
                        else:
                            num_13 = 0
                            

                    else:
                        num_13 = 0
                        den_13 = 1


                elif group.Claim_Rate_Type.values[j] == pathwayStr and (group.Claim_Status.values[j] in valid_claims): # denominator excludes pathway outcomes
              
                    den_13 = 0
                    df_13['Num_13'] = [num_13]
                    df_13['Den_13'] = den_13
                    break
                       
                 

                elif group.Claim_Rate_Type.values[j] == noClaim: 
                    if exitDate < group.CLAIM_AVAILABLE_DATE.values[j]:
                        num_13 = 0
                       
                    else:
                        
                        num_13 += 0 
                        den_13 += 1

                else: # either 26, 52 week claim
                    
                    num_13 = 0
                   
                       
                
        
        j += 1

    if den_13 > 1:
        den_13 = 1
        
    df_13['Num_13'] = [num_13]
    df_13['Den_13'] = den_13
    return df_13
 






In [484]:
### Continuous 
jobEntry = g.get_group(2738240909)
get_13_outcome(jobEntry, num_codes, den_codes, com_codes)

,Job_Seeker_ID,Site_Name,Contract_ID,specialist_site_type_code,Num_13,Den_13
0,2738240909,AimBig Employment WOY WOY,0212812K,NaN,1,1


In [485]:
jobEntry.Claim_Rate_Type.value_counts()

na                       30
DES 4wk Outcome           5
DES 13wk Full Outcome     5
Name: Claim_Rate_Type, dtype: int64

In [486]:


dups_color = df_pre.pivot_table(columns=['Job_Seeker_ID'], aggfunc='size')
print (dups_color)
# need to drop duplicates such that Job_Seeker_ID is unique: 


Job_Seeker_ID
61780         12
82960         16
85140         16
88290         16
95360         20
              ..
9981431809    21
9983739909    16
9985676409    16
9996900006    80
9999230109    10
Length: 3363, dtype: int64


### Aggregate 13_week_outcome function

Looping through df_comb and passing each group to the get_13_outcome function

In [487]:
# Number of distinct Job_Seeker_IDs from df_pre:
distinct = df_pre['Job_Seeker_ID'].unique()
distinct

array([9386301003, 8653691003, 7870290004, ..., 6688566909,   13302780,
        581616609], dtype=int64)

In [488]:
# adds outcome to df_13 ->

def agg_outcomes_13(df_pre, g, num_codes, den_codes, com_codes):
    df_13 = pd.DataFrame(columns= ['Job_Seeker_ID', 'Num_13', 'Den_13', 'Site_Name', 'Contract_ID', 'specialist_site_type_code']) # empty
    # takes grouped by job_seeker_id dataframe, passes each row to get_no#_outcome

    ids = df_pre['Job_Seeker_ID'].unique()
    for id in ids:
        if math.isnan(id) or id == 0:
            pass
        else:
            df_entry = g.get_group(id)
            f = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
            df_13 = df_13.append(f)
            

    return df_13 # returns output from get_13_outcome 

# map outcomes to dataframe -> one pass


x = agg_outcomes_13(df_pre, g, num_codes, den_codes, com_codes)

In [489]:
x

,Job_Seeker_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code
0,9386301003,0,1,AimBig Employment WOY WOY,0212812K,NaN
0,8653691003,0,1,AimBig Employment FAIRFIELD,0212899A,AALL
0,7870290004,0,1,AimBig Employment NORTH LAKES,0212795D,AALL
0,6498977509,0,1,AimBig Employment LEICHHARDT,0212845J,AALL
0,6638331004,0,1,AimBig Employment NORTH LAKES,0212795D,AALL
...,...,...,...,...,...,...
0,7495948709,0,1,AimBig Employment MOUNT DRUITT,0212853J,MENH
0,5553772007,0,1,AimBig Employment INALA,0212796E,NaN
0,6688566909,0,0,AimBig Employment Pty Ltd ST ALBANS,0212822B,MENH
0,13302780,0,0,AimBig Employment FAIRFIELD,0212844H,NaN


In [490]:
x = x.drop(columns=['Job_Seeker_ID'])

In [491]:
x = x.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code'])['Num_13', 'Den_13'].sum()


In [492]:
x

Num_13  \
Site_Name                      Contract_ID specialist_site_type_code           
AimBig Employment AITKENVALE   0212804K    MENH                            1   
                               0212805A    MUSK                            2   
AimBig Employment ALTONA NORTH 0212822B    MENH                            1   
                               0212823C    MUSK                            1   
AimBig Employment AUBURN       0212841E    AALL                            0   
...                                                                      ...   
AimBig Employment WERRIBEE     0212822B    MENH                            1   
                               0212823C    MUSK                            0   
AimBig Employment WOY WOY      0212812K    AALL                            3   
                               0212881B    AALL                            1   
Aimbig Employment LAUNCESTON   0212893F    AALL                            1   

                                                                      Den_13  
Site_Name                      Contract_ID specialist_site_type_code          
AimBig Employment AITKENVALE   0212804K    MENH                            4  
                               0212805A    MUSK                            2  
AimBig Employment ALTONA NORTH 0212822B    MENH                            2  
                               0212823C    MUSK                            1  
AimBig Employment AUBURN       0212841E    AALL                            5  
...                                                                      ...  
AimBig Employment WERRIBEE     0212822B    MENH                            7  
                               0212823C    MUSK                            2  
AimBig Employment WOY WOY      0212812K    AALL                            3  
                               0212881B    AALL                            2  
Aimbig Employment LAUNCESTON   0212893F    AALL                            1  

[142 rows x 2 columns]

In [493]:
x['Den_13'].sum()

479

### This numerator is closer -> specialist_site_type_code is NaN -> should rename to AALL?

In [494]:
x

Num_13  \
Site_Name                      Contract_ID specialist_site_type_code           
AimBig Employment AITKENVALE   0212804K    MENH                            1   
                               0212805A    MUSK                            2   
AimBig Employment ALTONA NORTH 0212822B    MENH                            1   
                               0212823C    MUSK                            1   
AimBig Employment AUBURN       0212841E    AALL                            0   
...                                                                      ...   
AimBig Employment WERRIBEE     0212822B    MENH                            1   
                               0212823C    MUSK                            0   
AimBig Employment WOY WOY      0212812K    AALL                            3   
                               0212881B    AALL                            1   
Aimbig Employment LAUNCESTON   0212893F    AALL                            1   

                                                                      Den_13  
Site_Name                      Contract_ID specialist_site_type_code          
AimBig Employment AITKENVALE   0212804K    MENH                            4  
                               0212805A    MUSK                            2  
AimBig Employment ALTONA NORTH 0212822B    MENH                            2  
                               0212823C    MUSK                            1  
AimBig Employment AUBURN       0212841E    AALL                            5  
...                                                                      ...  
AimBig Employment WERRIBEE     0212822B    MENH                            7  
                               0212823C    MUSK                            2  
AimBig Employment WOY WOY      0212812K    AALL                            3  
                               0212881B    AALL                            2  
Aimbig Employment LAUNCESTON   0212893F    AALL                            1  

[142 rows x 2 columns]

### separating the sub_types from the performance measure 


In [495]:
df_13_new = pd.DataFrame(columns=['Job_Seeker_ID', 'Num_13_full', 'Den_13_full'])

for index, row in df_13_new.iterrows():
    
    g_key = row['Job_Seeker_ID']
    g_group = g_supp.get_group(g_key)
    g_claims = g_group.CLAIM_RATE_TYPE.values
    if 'DES 13wk Full Outcome' in g_claims:
        #   get the relevant dataframe matches
            row_match = df_13[df_13['Job_Seeker_ID'] == g_key]
            Num_13_old = row_match['Num_13']
            Den_13_old = row_match['Den_13']
            data = pd.DataFrame({'Job_Seeker_ID' : [g_key], 'Num_13_full': Num_13_old, 'Den_13_full' : Den_13_old})
            df_13_new = df_13_new.append(data) 





### Pre-Quarterisation Output File:

In [496]:
### Testing the aggregate function for 13 outcomes
df_13 = agg_outcomes_13(df_pre, g, num_codes, den_codes, com_codes)

print('Number of IDs for 13 week outcomes: ' + str(len(df_13)))
df_13

Number of IDs for 13 week outcomes: 3363


,Job_Seeker_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code
0,9386301003,0,1,AimBig Employment WOY WOY,0212812K,NaN
0,8653691003,0,1,AimBig Employment FAIRFIELD,0212899A,AALL
0,7870290004,0,1,AimBig Employment NORTH LAKES,0212795D,AALL
0,6498977509,0,1,AimBig Employment LEICHHARDT,0212845J,AALL
0,6638331004,0,1,AimBig Employment NORTH LAKES,0212795D,AALL
...,...,...,...,...,...,...
0,7495948709,0,1,AimBig Employment MOUNT DRUITT,0212853J,MENH
0,5553772007,0,1,AimBig Employment INALA,0212796E,NaN
0,6688566909,0,0,AimBig Employment Pty Ltd ST ALBANS,0212822B,MENH
0,13302780,0,0,AimBig Employment FAIRFIELD,0212844H,NaN


In [497]:
# output file
output_str = 'C:/Users/AManalo/star_ratings_new/prediction_model/' + period + '/df_13_pre.csv'
df_13.to_csv(output_str)


### Post-Quarterisation for 13 Week Outcomes - Numerator doesn't change, only the DENOMINATOR
#### Check - The numerator achieved date -> 13 weeks after commencement date 
- Denominate each quarter if they do not numerate. For each quarter a participant is eligible to record an outcome, i.e. been in assistance at least 13 weeks - if no outcome is recorded, then the participant continues denominating. 
- If the quarter falls outside the 2 year period - record is not included
- not included if time to achieve an outcome < 13 weeks
- Need to break up the 2 year rolling period into quarters -> 8 quarters (Based on business year)


1. Given the 2 year rolling period - breaking the period up to quarters
Format: nested list [[start_date, end_date],...]
- 8 quarters


In [498]:
q_start_dates = pd.date_range(startDate, endDate, freq='BQS')
q_end_dates = pd.date_range(startDate, endDate, freq='BQ-DEC')
print("All the business quarterly BEGIN dates of ______:")
print(q_start_dates.values)
print("\nAll the business quarterly END dates of ______:")
print(q_end_dates.values)

print("\n")

All the business quarterly BEGIN dates of ______:
['2019-10-01T00:00:00.000000000' '2020-01-01T00:00:00.000000000'
 '2020-04-01T00:00:00.000000000' '2020-07-01T00:00:00.000000000'
 '2020-10-01T00:00:00.000000000' '2021-01-01T00:00:00.000000000'
 '2021-04-01T00:00:00.000000000' '2021-07-01T00:00:00.000000000']

All the business quarterly END dates of ______:
['2019-12-31T00:00:00.000000000' '2020-03-31T00:00:00.000000000'
 '2020-06-30T00:00:00.000000000' '2020-09-30T00:00:00.000000000'
 '2020-12-31T00:00:00.000000000' '2021-03-31T00:00:00.000000000'
 '2021-06-30T00:00:00.000000000' '2021-09-30T00:00:00.000000000']




In [499]:
id = df_13.Job_Seeker_ID.values[6]
i = 0
n = len(g.get_group(id))
group = g.get_group(id)
group.Claim_Rate_Type.values

if 'DES 13wk Full Outcome' in group.Claim_Rate_Type.values:
    print(group.Claim_Rate_Type.values)

outcomeInd = (group.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
outcomeInd


# while i < n:
    # out = group.Claim_Rate_Type.values[i]
    # print(out)
    # print(dt)
    # i += 1

0

Quarters:
Q1: '2020-10-01', '2020-12-31'
Q2: '2021-01-01', '2021-03-31'
Q3: '2021-04-01', '2021-06-30'
Q4: '2021-07-01', '2021-09-30'
Q5: '2021-10-01', '2021-12-31'
Q6: '2022-01-03', '2022-03-31'
Q7: '2022-04-01', '2022-06-30'
Q8: '2022-07-01', '2022-09-30'

In [500]:
qtrs = list(zip(q_start_dates.values, q_end_dates.values))

z = 0
s = 0
e = 1
while z < len(qtrs):
    if isinstance(qtrs[z][s], str):
        start = datetime.strptime(qtrs[z][s], '%Y-%m-%d')
        qtrs[z][s] = start
    if isinstance(qtrs[z][e], str):
        end = datetime.strptime(qtrs[z][e], '%Y-%m-%d')
        qtrs[z][e] = end
    z += 1

### Edge Cases for Post Quarterisation:
- If participant does not record a 13 week outcome ever -> denominates the max 8 times.
- If time to achieve an outcome is less than 13 weeks -> don't denominate. -> but this should be in the df_13 already 
- If quarter falls outside of the two year assessment period, then not included.


In [501]:
qNo = 0
while qNo < len(qtrs):
    s = (qtrs[qNo][0], "%Y-%m-%d") # start date of quarter
    e = (qtrs[qNo][1], "%Y-%m-%d")  # end date of quarter
    qNo += 1


i = 0
id = df_13.Job_Seeker_ID.values
Den_13 = 0 
Num_13 = 0
while i < len(id):
    id2 = id[i]
    
    x = 0
    group = g.get_group(id2)
    z = 0
    while z < len(group.CLAIM_AVAILABLE_DATE.values):
        
        claim = group.Claim_Rate_Type.values[z] 
        if claim == 'DES 13wk Full Outcome':  # first check for the type of claim rate -> find 13 week outcome

         
            Num_13 = 1
            # final denominator -> assign to current count of denominator -> ask if they denominate during the numerating
            Den_13 += 1 
            break
       
       
        else:
            Den_13 += 1
            pass 
            # just count through the denominators when the claim exists and is recorded
        z += 1
        
    i += 1


In [502]:
df_pre.dtypes

Job_Seeker_ID                          int64
REFERRAL_PHASE_CODE                   object
PLACEMENT_STATUS_CODE                 object
PLACEMENT_TYPE                        object
Program                               object
Contract_ID                           object
Site_Name                             object
ESA_Code                              object
specialist_site_type_code             object
COMMENCEMENT_DATE             datetime64[ns]
EXIT_DATE                             object
Claim_ID                               int64
Claim_Rate_Type                       object
Claim_Status                          object
CLAIM_START_DATE              datetime64[ns]
CLAIM_AVAILABLE_DATE          datetime64[ns]
CLAIM_END_DATE                datetime64[ns]
SUSPENSION_FROM_DATE          datetime64[ns]
SUSPENSION_ACTUAL_END_DATE    datetime64[ns]
OUTCOME_TYPE                          object
SUSPENSION_DAYS                        int64
PLACE_REF                             object
dtype: obj

In [503]:
df_pre['Claim_Rate_Type'].values

array(['na', 'na', 'na', ..., 'DES 26wk Full Outcome Education',
       'DES 26wk Full Outcome Education',
       'DES 26wk Full Outcome Education'], dtype=object)

In [504]:
df_13

,Job_Seeker_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code
0,9386301003,0,1,AimBig Employment WOY WOY,0212812K,NaN
0,8653691003,0,1,AimBig Employment FAIRFIELD,0212899A,AALL
0,7870290004,0,1,AimBig Employment NORTH LAKES,0212795D,AALL
0,6498977509,0,1,AimBig Employment LEICHHARDT,0212845J,AALL
0,6638331004,0,1,AimBig Employment NORTH LAKES,0212795D,AALL
...,...,...,...,...,...,...
0,7495948709,0,1,AimBig Employment MOUNT DRUITT,0212853J,MENH
0,5553772007,0,1,AimBig Employment INALA,0212796E,NaN
0,6688566909,0,0,AimBig Employment Pty Ltd ST ALBANS,0212822B,MENH
0,13302780,0,0,AimBig Employment FAIRFIELD,0212844H,NaN


In [505]:
group = g.get_group(2997015709)
group.EXIT_DATE.values

array(['2021-07-05 00:00:00', '2021-07-05 00:00:00',
       '2021-07-05 00:00:00', '2021-07-05 00:00:00',
       '2021-07-05 00:00:00', '2021-07-05 00:00:00',
       '2021-07-05 00:00:00', '2021-07-05 00:00:00',
       '2021-07-05 00:00:00', '2021-07-05 00:00:00',
       '2021-07-05 00:00:00', '2021-07-05 00:00:00',
       '2021-07-05 00:00:00', '2021-07-05 00:00:00',
       '2021-07-05 00:00:00', '2021-07-05 00:00:00',
       '2021-07-05 00:00:00', '2021-07-05 00:00:00',
       '2021-07-05 00:00:00', '2021-07-05 00:00:00'], dtype=object)

For 13 week pathway outcomes:
- If this type of outcome is recorded, it is not included in the denominator -> ignored
- However if a participant is at the site for at least 13 weeks and no FULL outcome is recorded -> continues denominating until full outcome is recorded 

In [506]:
qtrs
endBounds = endDate
startBounds = startDate

def post_quarter_13(df_13, df_pre, g, qtrs):
    '''Post quarterisation performance for 13 week full outcomes. Receives df_pre as an input, and outputs the post_quarterised denominators for each eligible participant.
    df_13 is used to check if a participant has numerated, df_pre is used to feed in the job_seeker_id to 'g' - the groupBy object and check whether the job seeker denominates - if no numerator in the df_13 exists.'''
    # maximum amount of times they numerate/denominate:
    maxDen = 8
    maxNum = 1

    df_post = pd.DataFrame(columns=['Job_Seeker_ID', 'Num_13', 'Den_13', 'Site_Name', 'Contract_ID', 'specialist_site_type_code']) # Dataframe template for result
    
    claimExists = False
    
    i = 0
    while i < len(df_13):
       
        js_id = df_13.Job_Seeker_ID.values[i]
        siteName = df_13.Site_Name.values[i]
        contractID = df_13.Contract_ID.values[i]
        specialistCode = df_13.specialist_site_type_code.values[i]
        num_13 = df_13.Num_13.values[i] 
        den_13 = df_13.Den_13.values[i]
        Den_13 = 0
    
        Num_13 = num_13
        


    

    
        
        x = 0
        group = g.get_group(js_id)
        n = len(group)
        week_13 = pd.DateOffset(days = 91)
        stopLoop = False
        while x < n or not stopLoop:
            comDate = pd.to_datetime(group.COMMENCEMENT_DATE.values[0], format='%Y-%m-%d') # commencement date should be the first index 
            claim = group.Claim_Rate_Type.values[x]
            availDate = group.CLAIM_AVAILABLE_DATE.values[x]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[x]) # exit date should be recorded FOR ALL
            invalid_date = pd.to_datetime('2001-01-01', format='%Y-%m-%d')

          
            if den_13 == maxDen:
                pass # return -> denominated maxDen = 8 times, or maxNum/maxDen
            
            if num_13 == 0 and den_13 == 0:
                Num_13 = 0
                Den_13 = 0
                stopLoop = True
                break 
            
        
            elif comDate <= invalid_date:
                Num_13 = 0
                Den_13 += 0
            else: 
                # looping through the outcomes
                if (group.PLACEMENT_STATUS_CODE.values[x] == 'Exited') and (group.SUSPENSION_DAYS.values[x] > 0): # current entry is showing Exited -> check if exitDate is  
                    period_13 = comDate + week_13 
                    if (group.SUSPENSION_FROM_DATE.values[x] >= comDate) and (group.SUSPENSION_ACTUAL_END_DATE.values[x] <= period_13): 
                        # then suspension is applicable to this outcome and period
                        period_13 = comDate + week_13 + pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[x]))
                       
                        # check if participant has exited before the 13 week outcome was due: 
                        if exitDate < period_13: 
                            # then don't include at all -> wipe all numerator/denominator
                            Num_13 = 0 
                            Den_13 += 0
                            pass
                    
                    

                # loop through the quarters and compare with claim_available_date -> check which quarter it occurs in
                else:
                    
                    if (claim == 'DES 13wk Full Outcome') and (group.Claim_Status.values[x] == 'Approved'): # if 13 week outcome claim exists - validate the number for the denominator -> reset
                        Den_13 = 1 
                    
                        Num_13 = 1
                        pass
                        
                    elif claim == 'DES 52wk Full Outcome' or claim == 'DES 26wk Full Outcome':
                        Num_13 += 0 
                        Den_13 += 0 
                        pass
                    # 13 week outcome may already be recorded -> or may be in later entries
                    
                    else:
                        period_13 = comDate + week_13
                    # check if a 13 week full outcome exists in ANY of the group values
                        
                    z = 0
                    s = 0
                    e = 1
                    while z < len(qtrs) and not stopLoop:
                        start = pd.to_datetime(qtrs[z][s])
                        end = pd.to_datetime(qtrs[z][1])
                        
                        if z == len(qtrs) - 1:  # This is the last iteration
                            stopLoop = True
                        if Den_13 == maxDen or Num_13 == maxNum:
                            stopLoop = True
                            break
                        else:
                            if 'DES 13wk Full Outcome' in group.Claim_Rate_Type.values:
                                # if the iteration -> loop is at the beginning of the current group -> get the commencement date of the FIRST row for the 13 week full outcome: 
                                outcomeInd = (group.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                                comDate = group.COMMENCEMENT_DATE.values[outcomeInd]
                                # get EXIT date for this here
                        
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[outcomeInd], format='%Y-%m-%d') # exit date should be recorded FOR ALL
                                ## TO-DO - investigate whether a SUSPENSION period occurs here and if exitDate <= period_13 (comDate + suspension_period + 13 weeks)
                            elif ('DES 13wk Full Outcome' not in group.Claim_Rate_Type.values ) and ('DES 13wk Pathway Outcome' in group.Claim_Rate_Type.values):    # all values are 13 week PATHWAY outcomes
                                pathwayInd = (group.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                                comDate = group.COMMENCEMENT_DATE.values[pathwayInd]
                    
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[pathwayInd], format='%Y-%m-%d' ) # exit date should be recorded FOR ALL
                            # check here how many times the participant denominates ->
                            if (comDate >= start) and (comDate <= end) and (group.PLACEMENT_STATUS_CODE.values[x] == 'Commenced'): # check if the commencement date is in the current quarter: 
                                
                                period_13 = comDate + week_13 # denominates if and only if at least 13 weeks has passed: 13 weeks + comDate -> falls into the current quarter, if not pass
                                if claim == 'DES 13wk Pathway Outcome':
                                    Den_13 = 0
                            
                                elif (exitDate >= start) and (exitDate <= end) and (period_13 >= start) and (period_13 <= end): #exit date is in the current quarter -> then denominates and then exists
                                
                                    Den_13 += 1 
                                    stopLoop = True
                                    break
                                elif (availDate >= start) and (availDate <= end) and (period_13 >= start) and (period_13 <= end): # in the current quarter -> exit date not counted
                                    Den_13 += 1
                                   
                                #add here if exit before period_13
                                elif (exitDate < period_13):
                                
                                    Den_13 = 0
                                    stopLoop = True
                                    break
                            


                            else: # commencement date has already passed, OR commencement date is later
                                # need to stop at participant EXIT_DATE - get exit date of the 13 week full outcome or pathway outcome:
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[0], format='%Y-%m-%d' ) # exit date should be recorded FOR ALL
                                period_13 =  comDate + week_13
                                
                                if (comDate > end): # commencement date is later ->
                                    pass
                                    
                                elif exitDate < period_13:
                     
                                    Den_13 = 0
                                    stopLoop = True
                                    break 
                                
                                else: 
                               
                                    # check if exitDate is within the current quarter being traversed through
                                    if (exitDate >= start) and (exitDate <= end):
                                        if period_13 >= start and period_13 <= end: # duration at least 13 weeks OCCURS during this quarter and they exit in this quarter
                                            Den_13 += 1
                                            stopLoop = True
                                            break
                                        elif exitDate < period_13:
                                            Den_13 = 0
                                            stopLoop = True
                                            break
                                        else:
                                
                                            Den_13 += 1
                                            stopLoop = True
                                            break
                                            
                                    elif (period_13 >= start and period_13 <= end) or (period_13 < start) and not (start <= exitDate <= end):
                                        
                                        Den_13 += 1

                    
                                       
                                    else:
                                        Den_13 += 0
                                       
                                
                        z += 1
                        
                   
                        
                    
                

                  

                x += 1
        

        if Den_13 >= maxDen:
            Den_13 = 8  # set threshold
        data = pd.DataFrame({'Job_Seeker_ID' : [js_id], 'Num_13_full' : num_13, 'Den_13_full' : Den_13, 'Site_Name' : [siteName], 'Contract_ID' : [contractID], 'specialist_site_type_code' : [specialistCode]})
        df_post = df_post.append(data)

            

            
        
        i += 1
            





    return df_post # returns post-quarterisation dataframe 




In [507]:
qtrs[2][0]

numpy.datetime64('2020-04-01T00:00:00.000000000')

In [508]:

x = agg_outcomes_13(df_pre, g, num_codes, den_codes, com_codes)
df_13_post = post_quarter_13(x, df_pre, g, qtrs)

output_str = 'C:/Users/AManalo/star_ratings_new/prediction_model/' + period + '/df_13_post.csv'
df_13_post.to_csv(output_str)
df_13_post = df_13_post.groupby(['Job_Seeker_ID', 'Site_Name', 'Contract_ID', 'specialist_site_type_code'])['Num_13_full', 'Den_13_full'].sum()
df_13_post = df_13_post.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code'])['Num_13_full', 'Den_13_full'].sum()
df_13_post

Num_13_full  \
Site_Name                      Contract_ID specialist_site_type_code                
AimBig Employment AITKENVALE   0212804K    MENH                               1.0   
                               0212805A    MUSK                               2.0   
AimBig Employment ALTONA NORTH 0212822B    MENH                               1.0   
                               0212823C    MUSK                               1.0   
AimBig Employment AUBURN       0212841E    AALL                               0.0   
...                                                                           ...   
AimBig Employment WERRIBEE     0212822B    MENH                               1.0   
                               0212823C    MUSK                               0.0   
AimBig Employment WOY WOY      0212812K    AALL                               3.0   
                               0212881B    AALL                               1.0   
Aimbig Employment LAUNCESTON   0212893F    AALL                               1.0   

                                                                      Den_13_full  
Site_Name                      Contract_ID specialist_site_type_code               
AimBig Employment AITKENVALE   0212804K    MENH                               3.0  
                               0212805A    MUSK                               2.0  
AimBig Employment ALTONA NORTH 0212822B    MENH                               4.0  
                               0212823C    MUSK                               1.0  
AimBig Employment AUBURN       0212841E    AALL                              23.0  
...                                                                           ...  
AimBig Employment WERRIBEE     0212822B    MENH                               6.0  
                               0212823C    MUSK                               9.0  
AimBig Employment WOY WOY      0212812K    AALL                               3.0  
                               0212881B    AALL                               2.0  
Aimbig Employment LAUNCESTON   0212893F    AALL                               1.0  

[142 rows x 2 columns]

In [509]:
output_str = 'C:/Users/AManalo/star_ratings_new/prediction_model/' + period + '/13_week_outcomes_sites.csv'

df_13_post.to_csv(output_str)


In [510]:
df_13

,Job_Seeker_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code
0,9386301003,0,1,AimBig Employment WOY WOY,0212812K,NaN
0,8653691003,0,1,AimBig Employment FAIRFIELD,0212899A,AALL
0,7870290004,0,1,AimBig Employment NORTH LAKES,0212795D,AALL
0,6498977509,0,1,AimBig Employment LEICHHARDT,0212845J,AALL
0,6638331004,0,1,AimBig Employment NORTH LAKES,0212795D,AALL
...,...,...,...,...,...,...
0,7495948709,0,1,AimBig Employment MOUNT DRUITT,0212853J,MENH
0,5553772007,0,1,AimBig Employment INALA,0212796E,NaN
0,6688566909,0,0,AimBig Employment Pty Ltd ST ALBANS,0212822B,MENH
0,13302780,0,0,AimBig Employment FAIRFIELD,0212844H,NaN


# 26 Week Outcomes

### Eligible 26 week outcomes:
Included:
- Full 26 week outcomes (excluding Education - only vacancy )
- Pathway 26 week outcomes (worth 5%)
- Bonus/work assist outcomes 

Excluded:
- Zero dollar outcomes 
- Education outcomes

1) Renaming Num_26 and Den_26 => Num_path_26, Den_path_26, Num_wrkast_26, den_wrkast_26 (work assist/bonus), Num_26 -> Num_full_26, Den_full_26


### 1) Pre-Quarterisation

Difference to the existing pre-quarterisation function: 

    1.  Get the date from the [13 week outcome] claim_available_date - in which the 13 week outcome was achieved (not commencement_date like with 13 week outcome)
        done

    NOW: assuming that the data is correct - continue the pre-quarterisation function 


    2. Add 13 weeks to the date from (1) excluding suspension periods. Check if EXIT_DATE occurs before 26 week is recorded -> excluded. 

    3. If there exists a 26 week eligible outcome within the 2 year rolling period (after the 13_week_claim_available_date) –> NUM_26 = 1, DEN_26 = 1

    4. Otherwise: NUM_26 = 0, DEN_26 = 1

    5. If 26 week outcome ELIGIBILITY DATE (26 weeks from 13 week outcome) falls outside the assessment period, then -> NUM_26 = 0, DEN_26 = 0 

Confirm types of eligible 26 week outcomes to be taken <- from Chiara’s email. 





### Importing supp_apv_stream_data (extension of assessment period)
- extends assessment period by 6 months prior to the start date
- only checked if in the current assessment period, there exists 26 or 52 week outcome and NO prior outcome to compare with. 
- pre-process the file - to ensure data is of the correct format

In [511]:
df_supp = pd.read_csv("C:/Users/AManalo/OneDrive - Arriba Group/Desktop/Star Ratings - AimBig/supp_apv_stream_data.csv") 
# rename columns to capital:
df_supp.columns = map(str.upper, df_supp.columns)
df_supp.dtypes

COMMENCEMENT_DATE                    object
PROGRAM_TYPE_CODE                    object
JR_JOBSEEKER_ID                       int64
JOB_SEEKER_ID                         int64
PLAC_JOBSEEKER_ID                     int64
PLACEMENT_TYPE                       object
PROGRAM                              object
CONTRACT_ID                          object
ESA_CODE                             object
SPECIALIST_SITE_TYPE_CODE            object
REFERRAL_DATE                        object
REFERRAL_PHASE_CODE                  object
PLACEMENT_STATUS_CODE                object
PHASE_CHANGED                       float64
STATUS_CHANGED                        int64
PHASE_COMMENCEMENT_DATE              object
TIME_IN_PROGRAM_WEEKS               float64
PERIOD_OF_SERVICE_WEEKS             float64
DISABILITY_TYPE                      object
MODERATE_INTELLECTUAL_DISABILITY     object
INDIGENOUS                          float64
VOLUNTEER                              bool
JSCI_STATUS                     

--------------

### Pre-processing df_supp
1. Cleaning columns
2. Creating another groupBy object -> this is checked when the previous outcome (prior to the current 26/52 outcome) is missing in the 2Y period. 


In [512]:


# if both are NA -> then no anchor date



df_supp['COMMENCEMENT_DATE_NEW'] = np.where((df_supp['PPS_START_DATE'].isna()) & (df_supp['OVERRIDDEN_OUTCOME_STARTED_ON'].isna()), None, None)


# PPS_START_DATE doesnt exist but overriden_outcome_started_on exists 
df_supp['COMMENCEMENT_DATE_NEW'] = np.where((df_supp['PPS_START_DATE'].isna()) & (df_supp['OVERRIDDEN_OUTCOME_STARTED_ON'].isna() == False), df_supp['OVERRIDDEN_OUTCOME_STARTED_ON'], None)
# overriden_outcome_started_on doesn't exists, but PPS_START_DATE existd
df_supp['COMMENCEMENT_DATE_NEW'] = np.where((df_supp['PPS_START_DATE'].isna() == False) & (df_supp['OVERRIDDEN_OUTCOME_STARTED_ON'].isna()), df_supp['PPS_START_DATE'], df_supp['COMMENCEMENT_DATE_NEW'])

# if both exist -> take overriden_outcome_started_on
df_supp['COMMENCEMENT_DATE_NEW'] = np.where((df_supp['PPS_START_DATE'].isna() == False) & (df_supp['OVERRIDDEN_OUTCOME_STARTED_ON'].isna() == False), df_supp['OVERRIDDEN_OUTCOME_STARTED_ON'], df_supp['COMMENCEMENT_DATE_NEW'])

# drop rows where COMMENCEMENT_DATE_NEW is NULL -> no valid placements
df_supp = df_supp[df_supp['COMMENCEMENT_DATE_NEW'].isna() == False]
# drop None
df_supp = df_supp[df_supp['COMMENCEMENT_DATE_NEW'] != None]

# dropping the original "COMMENCEMENT_DATE" ->
df_supp = df_supp.drop(columns=['COMMENCEMENT_DATE'])

df_supp = df_supp.rename(columns={'COMMENCEMENT_DATE_NEW' : 'COMMENCEMENT_DATE'})

# subset: 
df_supp = df_supp[["JOB_SEEKER_ID", 'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE', 'PLACEMENT_TYPE', 'COMMENCEMENT_DATE', 'EXIT_DATE', 'CLAIM_ID', 'CLAIM_RATE_TYPE', 'CLAIM_STATUS', 'CLAIM_START_DATE', 'CLAIM_AVAILABLE_DATE', 'CLAIM_END_DATE', 'SUSPENSION_FROM_DATE', 'SUSPENSION_ACTUAL_END_DATE', 'OUTCOME_TYPE']]


df_supp = df_supp.sort_values(by=['JOB_SEEKER_ID', 'CLAIM_START_DATE'])

# converting data types + converting NaT to out of range dates (i.e. 2001)
df_supp['JOB_SEEKER_ID'] = pd.to_numeric(df_supp['JOB_SEEKER_ID'], errors='coerce')
df_supp = df_supp[df_supp['JOB_SEEKER_ID'].isna() != True]
df_supp['JOB_SEEKER_ID'] = df_supp['JOB_SEEKER_ID'].astype('int64')



# need to convert column into string first -> then back to date
dict_types = {'SUSPENSION_FROM_DATE' : 'str', 'COMMENCEMENT_DATE' : 'str', 'CLAIM_START_DATE' : 'str', 'CLAIM_AVAILABLE_DATE' : 'str', 'CLAIM_END_DATE' : 'str', 'SUSPENSION_ACTUAL_END_DATE' : 'str'}
df_supp = df_supp.astype(dict_types)
fillVal = pd.to_datetime('2001-12-23', format='%Y-%m-%d')



df_supp['SUSPENSION_FROM_DATE'] = pd.to_datetime(df_supp['SUSPENSION_FROM_DATE'], format='%Y-%m-%d')
df_supp['SUSPENSION_FROM_DATE'] = df_supp['SUSPENSION_FROM_DATE'].fillna(fillVal)

df_supp['SUSPENSION_ACTUAL_END_DATE'] = pd.to_datetime(df_supp['SUSPENSION_ACTUAL_END_DATE'], format='%Y-%m-%d')
df_supp['SUSPENSION_ACTUAL_END_DATE'] = df_supp['SUSPENSION_ACTUAL_END_DATE'].fillna(fillVal)

df_supp['COMMENCEMENT_DATE'] = pd.to_datetime(df_supp['COMMENCEMENT_DATE'], format='%Y-%m-%d')
df_supp['COMMENCEMENT_DATE'] = df_supp['COMMENCEMENT_DATE'].fillna(fillVal)


df_supp['CLAIM_START_DATE'] = pd.to_datetime(df_supp['CLAIM_START_DATE'], format='%Y-%m-%d')
df_supp['CLAIM_START_DATE'] = df_supp['CLAIM_START_DATE'].fillna(fillVal)

df_supp['CLAIM_AVAILABLE_DATE'] = pd.to_datetime(df_supp['CLAIM_AVAILABLE_DATE'], format='%Y-%m-%d')
df_supp['CLAIM_AVAILABLE_DATE'] = df_supp['CLAIM_AVAILABLE_DATE'].fillna(fillVal)

df_supp['CLAIM_END_DATE'] = pd.to_datetime(df_supp['CLAIM_END_DATE'], format='%Y-%m-%d', errors='coerce')
df_supp['CLAIM_END_DATE'] = df_supp['CLAIM_END_DATE'].fillna(fillVal)




df_supp['COMMENCEMENT_DATE'] = df_supp["COMMENCEMENT_DATE"].loc[(df_supp['COMMENCEMENT_DATE'] <= endDate )]

df_supp["CLAIM_RATE_TYPE"] = df_supp["CLAIM_RATE_TYPE"].replace(np.nan, '', regex=True)

df_supp["CLAIM_RATE_TYPE"] = df_supp["CLAIM_RATE_TYPE"].astype('str')

# remove education outcome_type, or NaN outcome_type
df_supp = df_supp.loc[:,~df_supp.columns.duplicated()].copy()

# getting the period between suspension_from_date and suspension_actual_end_date in weeks
df_supp['SUSPENSION_DAYS'] = (df_supp['SUSPENSION_ACTUAL_END_DATE'] - df_supp['SUSPENSION_FROM_DATE']).apply(lambda x: x/np.timedelta64(1, 'D'))



df_supp['CLAIM_RATE_TYPE'].value_counts()



                                                      269846
DES 4wk Outcome                                        57142
DES 13wk Full Outcome                                  31730
DES 26wk Full Outcome                                  21705
DES 52wk Full Outcome                                   7727
DES 13wk Full Outcome Education                         2810
DES 26wk Full Outcome Education                         2648
DES 13wk Pathway Outcome                                2074
DES 26wk Pathway Outcome                                 598
DES 13wk Pathway Outcome Education                       500
DES 52wk Pathway Outcome                                 283
DES 4wk Outcome Moderate Intellectual Disability P        35
DES 13wk Outcome Moderate Intellectual Disability         28
DES 26wk Outcome Moderate Intellectual Disability         16
DES 13wk Full Outcome Zero Dollar R4                       8
DES 52wk Outcome Moderate Intellectual Disability          6
Name: CLAIM_RATE_TYPE, d

In [513]:
# combine the placement_status_code and referral_phase_code :
df_supp['PLACE_REF'] = list(zip(df_supp.PLACEMENT_STATUS_CODE, df_supp.REFERRAL_PHASE_CODE))

df_supp = df_supp[df_supp['CLAIM_AVAILABLE_DATE'].isna() == False ]
df_supp['CLAIM_AVAILABLE_DATE'] = pd.to_datetime(df_supp['CLAIM_AVAILABLE_DATE'])
df_supp['CLAIM_AVAILABLE_DATE'].min()


Timestamp('2001-12-23 00:00:00')

In [514]:
df_supp['CLAIM_AVAILABLE_DATE'].max()

Timestamp('2022-10-07 00:00:00')

In [515]:
df_supp['SUSPENSION_DAYS'] = np.where(df_supp['SUSPENSION_DAYS'].isna(), 0, df_supp['SUSPENSION_DAYS'])



### groupBy object -> "g_supp"

In [516]:
df_supp = df_supp.sort_values(['COMMENCEMENT_DATE', 'CLAIM_START_DATE'])
# drop duplicates:

g_supp = df_supp.groupby(["JOB_SEEKER_ID"]) # remove commencement_date with NaT -> check if NaT values, filter these out and remove


### Pre-quarterisation function (26 week outcome)

In [517]:
df_entry = g.get_group(451780)
df_entry

,Job_Seeker_ID,REFERRAL_PHASE_CODE,PLACEMENT_STATUS_CODE,PLACEMENT_TYPE,Program,Contract_ID,Site_Name,ESA_Code,specialist_site_type_code,COMMENCEMENT_DATE,...,Claim_Rate_Type,Claim_Status,CLAIM_START_DATE,CLAIM_AVAILABLE_DATE,CLAIM_END_DATE,SUSPENSION_FROM_DATE,SUSPENSION_ACTUAL_END_DATE,OUTCOME_TYPE,SUSPENSION_DAYS,PLACE_REF
282903,451780,EA,Suspended,Vacancy Placement,Dms3,0212875D,AimBig Employment CALOUNDRA,4SUC,MUSK,2020-11-19,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Suspended, EA)"
282449,451780,PPS,Commenced,Vacancy Placement,Dms3,0212875D,AimBig Employment CALOUNDRA,4SUC,MUSK,2020-11-19,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Commenced, PPS)"
282448,451780,PPS,Commenced,Vacancy Placement,Dms3,0212875D,AimBig Employment CALOUNDRA,4SUC,MUSK,2020-11-19,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Commenced, PPS)"
269549,451780,EA,Commenced,Vacancy Placement,Dms3,0212875D,AimBig Employment CALOUNDRA,4SUC,MUSK,2020-11-19,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Commenced, EA)"
269548,451780,EA,Commenced,Vacancy Placement,Dms3,0212875D,AimBig Employment CALOUNDRA,4SUC,MUSK,2020-11-19,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Commenced, EA)"
266446,451780,OS,Exited,Vacancy Placement,Dms3,0212875D,AimBig Employment CALOUNDRA,4SUC,MUSK,2020-11-19,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Exited, OS)"
266445,451780,OS,Exited,Vacancy Placement,Dms3,0212875D,AimBig Employment CALOUNDRA,4SUC,MUSK,2020-11-19,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Exited, OS)"
258116,451780,EA,Pending,Vacancy Placement,Dms3,0212875D,AimBig Employment CALOUNDRA,4SUC,MUSK,2020-11-19,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Pending, EA)"
258102,451780,EA,Pending,Vacancy Placement,Dms3,0212875D,AimBig Employment CALOUNDRA,4SUC,MUSK,2020-11-19,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Pending, EA)"
283498,451780,EA,Suspended,Vacancy Placement,Dms3,0212875D,AimBig Employment CALOUNDRA,4SUC,MUSK,2020-11-19,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Suspended, EA)"


In [518]:
def get_26_outcome(jobSeekerEntry, num_codes, den_codes, com_codes): # takes a row (groupBy on Job_Seeker_ID, list of eligible numerator phase codes + denominator phase codes
    # sort jobSeekerEntry by COMMENCEMENT_DATE
    
    endBounds = endDate
    startBounds = startDate
    
    jobSeekerEntry = jobSeekerEntry.sort_values(by=['COMMENCEMENT_DATE'])
    group = jobSeekerEntry # group containing the attributes for one job_seeker_ID
    id = group.index.values.astype(int)[0]
    num_26_full = 0
    den_26_full = 0
    num_26_path = 0 
    den_26_path = 0
    num_26_wrkast = 0 
    den_26_wrkast = 0
    df_26 = pd.DataFrame()
    # TEMP ----------- REMOVE
    df_26['Site_Name'] = [group.Site_Name.values[0]]
    df_26['Contract_ID'] = [group.Contract_ID.values[0]]
    df_26['specialist_site_type_code'] = [group.specialist_site_type_code.values[0]]

    # TEMP -------------------
    # place id into dataframe 
    df_26['Job_Seeker_ID'] = [group.Job_Seeker_ID.values[0]]
    # initialise the variables in df_26
    df_26['Num_26_full'] = [num_26_full]
    df_26['Den_26_full'] = 0
    df_26['Num_26_path'] = [num_26_path]
    df_26['Den_26_path'] = 0
    df_26['Num_26_wrkast'] = [num_26_wrkast]
    df_26['Den_26_wrkast'] = 0

    week_13 = pd.DateOffset(days = 91)
    week_26 = pd.DateOffset(days = 182) # 26 weeks -> because adding from commencement_date -> but maybe it should be from 13 week ?
    eduStr = ['DES 26wk Full Outcome Education']
    # Check number of entries in group
    targetStr = ['DES 26wk Full Bonus', 'DES 26wk Full Outcome', 'DES 26wk Pathway Outcome', 'DES 26wk Work Assist Outcome'] #valid outcomes
    num_entries = int(len(jobSeekerEntry))
    noClaim = 'na'

    valid_claims = ['Approved', 'Pending', 'Lodged'] 
    exitDate = group.EXIT_DATE.values[0] # if exit date exists ->  exit date is the same for all entries

    # now loop ->
    j = 0
    comDate =  group.COMMENCEMENT_DATE.values[0]
    exitDate = group.EXIT_DATE.values[0] 
    while j < len(group.COMMENCEMENT_DATE.values):
        # commencement check ->
        if group.PLACEMENT_STATUS_CODE.values[j] == 'Commenced':
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_26 = comDate + week_26
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension
                # check suspension days is in the 2 year assessment period:
                 if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_26): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_26 = comDate + week_26 + sus_days
            else:
                pass

            # claim rate types:
            if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                
                # check if participant exits before claim_available_date ->
            
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if (exitDate < period_26):
                    if comDate > exitDate: # if commenced again after exit date:
                        # get the claim_rate_type -> 
                        if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                            num_26_full = 1
                            den_26_full = 1
                            df_26['Num_26_full'] = [num_26_full]
                            df_26['Den_26_full'] = 1
                            return df_26
                        elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                            num_26_wrkast = 1
                            den_26_wrkast = 1
                            df_26['Num_26_wrkast'] = [num_26_wrkast]
                            df_26['Den_26_wrkast'] = 1
                            return df_26
                        elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                            num_26_path = 1
                            den_26_path = 1
                            df_26['Num_26_path'] = [num_26_path]
                            df_26['Den_26_path'] = 1
                            return df_26

                    else:
                        num_26_full = 0
                        den_26_full = 0
                        num_26_path = 0 
                        den_26_path = 0
                        num_26_wrkast = 0 
                        den_26_wrkast = 0

                else: # check claim type 
                    if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                        num_26_full = 1
                        den_26_full = 1
                        df_26['Num_26_full'] = [num_26_full]
                        df_26['Den_26_full'] = 1
                        return df_26
                    elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                        num_26_wrkast = 1
                        den_26_wrkast = 1
                        df_26['Num_26_wrkast'] = [num_26_wrkast]
                        df_26['Den_26_wrkast'] = 1
                        return df_26
                    elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                        num_26_path = 1
                        den_26_path = 1
                        df_26['Num_26_path'] = [num_26_path]
                        df_26['Den_26_path'] = 1
                        return df_26
            elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_26 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if exitDate < period_26: # duration in assistance is less than 26 weeks
                    if comDate > exitDate: # if commenced again after exit date:
                        num_26_full += 0
                        den_26_full = 1


                    else: #just exited before period_26 
                        num_26_full = 0
                        den_26_full = 0
                        num_26_path = 0 
                        den_26_path = 0
                        num_26_wrkast = 0 
                        den_26_wrkast = 0
                   
                else: # check duration in assistance is at least period_26
                    num_26_full += 0
                    den_26_full = 1 #education outcomes are counted in the denominator of the full outcome

            elif group.Claim_Rate_Type.values[j] == noClaim: 
                
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if exitDate < period_26: # duration in assistance is less than 26 weeks
                    num_26_full = 0
                    den_26_full = 0
                    num_26_path = 0 
                    den_26_path = 0
                    num_26_wrkast = 0 
                    den_26_wrkast = 0
                else: # just denominate in the full outcome
            
                    num_26_full += 0 
                    den_26_full = 1
      
            else: # either 13 or 52 week claim
                num_26_full = 0
                

        else: # has not commenced ->
            
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_26 = comDate + week_26
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension 
                # check suspension days is in the 2 year assessment period:
                if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_26): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_26 = comDate + week_26 + sus_days
                
            # check if code is SUS_EA: (Counted in denominator if completed more than 26 weeks with us. Includes suspension period)
            if group.PLACE_REF.values[j] == 'SUS_EA':
                if group.CLAIM_AVAILABLE_DATE.values[j] >= period_26:
                    if (group.Claim_Rate_Type.values[j] in targetStr) and (group.Claim_Status.values[j] in valid_claims):
                        
                        if (period_26 <= startBounds) or (period_26 >= endBounds):
                                num_26_full = 0 
                                den_26_full = 0 # out of bounds
                        else:
                            # Check the claim type ->
                            if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                                num_26_full = 1
                                den_26_full = 1
                                df_26['Num_26_full'] = [num_26_full]
                                df_26['Den_26_full'] = 1
                                return df_26
                            elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                                num_26_wrkast = 1
                                den_26_wrkast = 1
                                df_26['Num_26_wrkast'] = [num_26_wrkast]
                                df_26['Den_26_wrkast'] = 1
                                return df_26
                            elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                                num_26_path = 1
                                den_26_path = 1
                                df_26['Num_26_path'] = [num_26_path]
                                df_26['Den_26_path'] = 1
                                return df_26
                    else:
                  
                        if (period_26 <= startBounds) or (period_26 >= endBounds):
                            num_26_full = 0 
                            den_26_full = 0 # out of bounds
                        else:
                            if (group.Claim_Rate_Type.values[j] == 'DES 26wk Pathway Outcome'): #this outcome IS NOt in the denominator
                                den_26_full = 0 
                                den_26_path = 1
                            else: # check the claim types and denominate these
                                if group.Claim_Rate_Type.values[j] == targetStr[1]: # full outcome
                                    den_26_full = 1
                                elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]:
                                    den_26_wrkast = 1
                                else: # NA claim
                                    den_26_full = 1

                else: # no claim_availble date or claim available date is less than period_26
                    if group.CLAIM_AVAILABLE_DATE.values[j] < period_26 and group.Claim_Rate_Type.values[j] == 'na':
                        # check duration in assistance
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate > period_26: # exits after 26 weeks
                            num_26_full += 0
                            den_26_full = 1
                        else: # exits before 26 weeks
                            num_26_full = 0
                            den_26_full = 0
                    else: # could be suspended but still record a claim ->
                        comDate = group.COMMENCEMENT_DATE.values[j]
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        period_26 = comDate + week_26
                        # check suspensions
                        if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension
                            # check suspension days is in the 2 year assessment period:
                            if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_26): # suspension in valid period ->
                                sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                                period_26 = comDate + week_26 + sus_days
                        else:
                            pass
 
                       
                        # claim_available_date could be valid and occur before period_26:
                        # claim rate types:
                        if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                            # check if participant exits before claim_available_date ->
                            
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if (exitDate < period_26):
                             

                                if comDate > exitDate:    # check if commences AFTER exit again:
                                    if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                                        num_26_full = 1
                                        den_26_full = 1
                                        df_26['Num_26_full'] = [num_26_full]
                                        df_26['Den_26_full'] = 1
                                        return df_26
                                    elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                                        num_26_wrkast = 1
                                        den_26_wrkast = 1
                                        df_26['Num_26_wrkast'] = [num_26_wrkast]
                                        df_26['Den_26_wrkast'] = 1
                                        return df_26
                                    elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                                        num_26_path = 1
                                        den_26_path = 1
                                        df_26['Num_26_path'] = [num_26_path]
                                        df_26['Den_26_path'] = 1
                                        return df_26 



                                else: # just exits before period_26
                                    num_26_full = 0
                                    den_26_full = 0
                                    num_26_path = 0 
                                    den_26_path = 0
                                    num_26_wrkast = 0 
                                    den_26_wrkast = 0

                            else: # check claim type 
                                if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                                    num_26_full = 1
                                    den_26_full = 1
                                    df_26['Num_26_full'] = [num_26_full]
                                    df_26['Den_26_full'] = 1
                                    return df_26
                                elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                                    num_26_wrkast = 1
                                    den_26_wrkast = 1
                                    df_26['Num_26_wrkast'] = [num_26_wrkast]
                                    df_26['Den_26_wrkast'] = 1
                                    return df_26
                                elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                                    num_26_path = 1
                                    den_26_path = 1
                                    df_26['Num_26_path'] = [num_26_path]
                                    df_26['Den_26_path'] = 1
                                    return df_26
                        elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_26 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if exitDate < period_26: # duration in assistance is less than 26 weeks
                                if comDate > exitDate: # commences again after exiting
                                    
                                    den_26_full = 1
                                    df_26['Num_26_full'] = [num_26_full]
                                    df_26['Den_26_full'] = den_26_full
                                else:
                                    num_26_full = 0
                                    
                                
                            else: # check duration in assistance is at least period_26
                                num_26_full = 0
                                den_26_full = 1 #education outcomes are counted in the denominator of the full outcome

                        elif group.Claim_Rate_Type.values[j] == noClaim: 
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if exitDate < period_26: # duration in assistance is less than 26 weeks
                                num_26_full = 0
                                den_26_full = 0
                                num_26_path = 0 
                                den_26_path = 0
                                num_26_wrkast = 0 
                                den_26_wrkast = 0
                            else: # just denominate in the full outcome
                                num_26_full += 0 
                                den_26_full += 1
                        else: # either 14 or 52 week claim
                            num_26_full = 0
                            
                
                                        
            else: # TRN or EXT
                ext_tfn = ['Exited', 'Transferred']
                if group.PLACEMENT_STATUS_CODE.values[j] in ext_tfn:
                    exitDate = pd.to_datetime(group.EXIT_DATE.values[j])

                    if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                        # check if participant exits before claim_available_date ->
                        
                    
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if (exitDate < period_26):
                            if comDate > exitDate: # commenced after exit
                                if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                                    num_26_full = 1
                                    den_26_full = 1
                                    df_26['Num_26_full'] = [num_26_full]
                                    df_26['Den_26_full'] = 1
                                    return df_26
                                elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                                    num_26_wrkast = 1
                                    den_26_wrkast = 1
                                    df_26['Num_26_wrkast'] = [num_26_wrkast]
                                    df_26['Den_26_wrkast'] = 1
                                    return df_26
                                elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                                    num_26_path = 1
                                    den_26_path = 1
                                    df_26['Num_26_path'] = [num_26_path]
                                    df_26['Den_26_path'] = 1
                                    return df_26
                                
                            
                            else:
                                num_26_full += 0
                                den_26_full = 0
                                num_26_path = 0 
                                den_26_path = 0
                                num_26_wrkast = 0 
                                den_26_wrkast = 0

                        else: # check claim type 
                            if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                                num_26_full = 1
                                den_26_full = 1
                                df_26['Num_26_full'] = [num_26_full]
                                df_26['Den_26_full'] = 1
                                return df_26
                            elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                                num_26_wrkast = 1
                                den_26_wrkast = 1
                                df_26['Num_26_wrkast'] = [num_26_wrkast]
                                df_26['Den_26_wrkast'] = 1
                                return df_26
                            elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                                num_26_path = 1
                                den_26_path = 1
                                df_26['Num_26_path'] = [num_26_path]
                                df_26['Den_26_path'] = 1
                                return df_26
                    elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_26 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate < period_26: # duration in assistance is less than 26 weeks
                            if comDate > exitDate: 
                                num_26_full += 0
                                den_26_full = 1
                            else:
                                num_26_full += 0
                                den_26_full = 0
                                num_26_path = 0 
                                den_26_path = 0
                                num_26_wrkast = 0 
                                den_26_wrkast = 0
                        
                        else: # check duration in assistance is at least period_26
                            num_26_full = 0
                            den_26_full = 1 #education outcomes are counted in the denominator of the full outcome

                    elif group.Claim_Rate_Type.values[j] == noClaim: 
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate < period_26: # duration in assistance is less than 26 weeks
                            num_26_full = 0
                            den_26_full = 0
                            num_26_path = 0 
                            den_26_path = 0
                            num_26_wrkast = 0 
                            den_26_wrkast = 0
                        else: # just denominate in the full outcome
                            num_26_full += 0 
                            den_26_full = 1
                          
                    else: # either 14 or 52 week claim
                        num_26_full += 0
                
            
                
        j += 1

    df_26['Num_26_full'] = [num_26_full]
    df_26['Den_26_full'] = den_26_full
    df_26['Num_26_path'] = [num_26_path]        
    df_26['Den_26_path'] = den_26_full - num_26_full
    df_26['Num_26_wrkast'] = [num_26_wrkast]
    df_26['Den_26_wrkast'] = den_26_wrkast
    return df_26

df_entry = g.get_group(4019304019)

f = get_26_outcome(df_entry, num_codes, den_codes, com_codes) # pass results to a new dataframe and append each time
f





                    


,Site_Name,Contract_ID,specialist_site_type_code,Job_Seeker_ID,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,Den_26_wrkast
0,AimBig Employment AUBURN,0212898K,NaN,4019304019,1,1,0,0,0,0


# aggregation function for pre-quarter function:

In [519]:
# adds outcome to df_26
def agg_outcomes_26(df_pre, g, num_codes, den_codes, com_codes):
    df_26 = pd.DataFrame(columns= ['Job_Seeker_ID', 'Num_26', 'Den_26', 'Site_Name', 'Contract_ID', 'specialist_site_type_code']) # empty
    # takes grouped by job_seeker_id dataframe, passes each row to get_no#_outcome



    ids = df_pre["Job_Seeker_ID"].unique().astype("int64")
    for id in df_pre['Job_Seeker_ID'].unique():
        if math.isnan(id):
            pass
        else:
            df_entry = g.get_group(id)
            f = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
            df_26 = df_26.append(f)
            

    return df_26 # returns output from get_26_outcome 

### Pre-quarterisation 26 output file:

In [520]:
### Testing the aggregate function for 13 outcomes
df_26 = agg_outcomes_26(df_pre, g, num_codes, den_codes, com_codes)


warnings.filterwarnings('ignore')

print('Number of IDs for 26 week outcomes: ' + str(len(df_26)))

# output file: 
output_str = 'C:/Users/AManalo/star_ratings_new/prediction_model/' + period + '/df_26_new.csv'
df_26.to_csv(output_str)


Number of IDs for 26 week outcomes: 3363


In [521]:
df_26['Den_26'].sum()

0

In [522]:
df_26

,Job_Seeker_ID,Num_26,Den_26,Site_Name,Contract_ID,specialist_site_type_code,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,Den_26_wrkast
0,9386301003,NaN,NaN,AimBig Employment WOY WOY,0212812K,NaN,0.0,1.0,0.0,1.0,0.0,0.0
0,8653691003,NaN,NaN,AimBig Employment FAIRFIELD,0212899A,AALL,0.0,1.0,0.0,1.0,0.0,0.0
0,7870290004,NaN,NaN,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,1.0,0.0,1.0,0.0,0.0
0,6498977509,NaN,NaN,AimBig Employment LEICHHARDT,0212845J,AALL,0.0,0.0,0.0,0.0,0.0,0.0
0,6638331004,NaN,NaN,AimBig Employment NORTH LAKES,0212795D,AALL,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,7495948709,NaN,NaN,AimBig Employment MOUNT DRUITT,0212853J,MENH,0.0,1.0,0.0,1.0,0.0,0.0
0,5553772007,NaN,NaN,AimBig Employment INALA,0212796E,NaN,0.0,1.0,0.0,1.0,0.0,0.0
0,6688566909,NaN,NaN,AimBig Employment Pty Ltd ST ALBANS,0212822B,MENH,0.0,0.0,0.0,0.0,0.0,0.0
0,13302780,NaN,NaN,AimBig Employment FAIRFIELD,0212844H,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [523]:
df_26_grouped = df_26.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code', 'Job_Seeker_ID']).agg({
    'Num_26_full': 'sum',
    'Den_26_full': 'sum',
    'Num_26_path': 'sum',
    'Den_26_path': 'sum',
    'Num_26_wrkast': 'sum',
    'Den_26_wrkast': 'sum'
})

df_26_grouped = df_26_grouped.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code']).agg({
    'Num_26_full': 'sum',
    'Den_26_full': 'sum',
    'Num_26_path': 'sum',
    'Den_26_path': 'sum',
    'Num_26_wrkast': 'sum',
    'Den_26_wrkast': 'sum'
})

df_26_grouped['Den_26_path'] = df_26_grouped['Den_26_full'] - df_26_grouped['Num_26_full'] 

In [524]:
df_26_temp = df_26_grouped

In [525]:
df_26_grouped = df_26_grouped.drop(columns=['Num_26_full', 'Den_26_full', 'Num_26_path', 'Num_26_wrkast', 'Den_26_wrkast'])

In [526]:
df_concat = pd.concat([df_26_grouped, df_26_temp])
df_concat

Den_26_path  \
Site_Name                      Contract_ID specialist_site_type_code                
AimBig Employment AITKENVALE   0212804K    MENH                               2.0   
                               0212805A    MUSK                               1.0   
AimBig Employment ALTONA NORTH 0212822B    MENH                               1.0   
                               0212823C    MUSK                               1.0   
AimBig Employment AUBURN       0212841E    AALL                               5.0   
...                                                                           ...   
AimBig Employment WERRIBEE     0212822B    MENH                               6.0   
                               0212823C    MUSK                               2.0   
AimBig Employment WOY WOY      0212812K    AALL                               0.0   
                               0212881B    AALL                               1.0   
Aimbig Employment LAUNCESTON   0212893F    AALL                               1.0   

                                                                      Num_26_full  \
Site_Name                      Contract_ID specialist_site_type_code                
AimBig Employment AITKENVALE   0212804K    MENH                               NaN   
                               0212805A    MUSK                               NaN   
AimBig Employment ALTONA NORTH 0212822B    MENH                               NaN   
                               0212823C    MUSK                               NaN   
AimBig Employment AUBURN       0212841E    AALL                               NaN   
...                                                                           ...   
AimBig Employment WERRIBEE     0212822B    MENH                               0.0   
                               0212823C    MUSK                               0.0   
AimBig Employment WOY WOY      0212812K    AALL                               3.0   
                               0212881B    AALL                               1.0   
Aimbig Employment LAUNCESTON   0212893F    AALL                               0.0   

                                                                      Den_26_full  \
Site_Name                      Contract_ID specialist_site_type_code                
AimBig Employment AITKENVALE   0212804K    MENH                               NaN   
                               0212805A    MUSK                               NaN   
AimBig Employment ALTONA NORTH 0212822B    MENH                               NaN   
                               0212823C    MUSK                               NaN   
AimBig Employment AUBURN       0212841E    AALL                               NaN   
...                                                                           ...   
AimBig Employment WERRIBEE     0212822B    MENH                               6.0   
                               0212823C    MUSK                               2.0   
AimBig Employment WOY WOY      0212812K    AALL                               3.0   
                               0212881B    AALL                               2.0   
Aimbig Employment LAUNCESTON   0212893F    AALL                               1.0   

                                                                      Num_26_path  \
Site_Name                      Contract_ID specialist_site_type_code                
AimBig Employment AITKENVALE   0212804K    MENH                               NaN   
                               0212805A    MUSK                               NaN   
AimBig Employment ALTONA NORTH 0212822B    MENH                               NaN   
                               0212823C    MUSK                               NaN   
AimBig Employment AUBURN       0212841E    AALL                               NaN   
...                                                                           ...   
AimBig Employment WERRIBEE     0212822B    MENH                         

## Post-Quarterisation 26 week outcome
    - Not included if time to achieve an outcome is < 26 weeks. AKA commenced towards the end of the 2Y assessment period.
    - Using the quarter dates from the 13 week pre-quarterisation function 
    - maxNum = 1, maxDen = 8
    - Needs to commence before denominating.
    - If df_26 (pre-quarter), num=0 -> then only check the denominator
    - If df_26 (pre-quarter), num=1 -> then need to check when it numerates
    - If quarter falls outside of the 2Y period, then not included
Included: 
-	26 week Pathway Outcomes
-	26 week Full Outcome
o	With lodged, pending or approved status at the site 
-	26 week Bonus Outcomes/Work Assist 

Denominator: 
- In order to denominate: duration in assistance it at least 26 weeks - excluding periods of suspension.
- EXCEPTION: if ANCHOR_DATE = CLAIM_AVAILABLE_DATE (for 13 week outcome).
    Method: 
    1. If the claims in the current JobSeeker group are ONLY 26/52 week outcomes -> check for 13 week outcome in the df_supp. 
    2. Get 13 week claim (Full outcome) from df_supp. 
    3. Check if ANCHOR_DATE = CLAIM_AVAILABLE_DATE (commencement_date) -> then only add 13 weeks to the ANCHOR_DATE for 26 week time in duration. 
    4. If ANCHOR_DATE != CLAIM_AVAILABLE_DATE, then check duration in assistance is at least 26 weeks (excluding suspensions) - add 13 weeks to 13_week CLAIM_AVAILABLE_DATE. -> don't need to check suspensions for the 13 week claim, as the claim has already been lodged/pending/approved. 
    - Check suspensions accordingly. 

- doesn't denominate if exits before 26 week duration in assistance


In [527]:
targetStr = ['DES 26wk Full Bonus', 'DES 26wk Full Outcome', 'DES 26wk Pathway Outcome', 'DES 26wk Work Assist Outcome'] #valid outcomes



Get the EXIT_DATE for the 26 wk claim -> to check whether job seeker exits before claim 

In [528]:
endBounds = endDate

startBounds = startDate

def post_quarter_26(df_26, df_pre, g, qtrs):
    '''Post quarterisation performance for 26 week full outcomes. Receives df_pre as an input, and outputs the post_quarterised denominators for each eligible participant.
    df_26 is used to check if a participant has numerated, df_pre is used to feed in the job_seeker_id to 'g' - the groupBy object and check whether the job seeker denominates - if no numerator in the df_26 exists.'''
    # maximum amount of times they numerate/denominate:
    maxDen = 8
    maxNum = 1

    df_post = pd.DataFrame(columns=['Job_Seeker_ID', 'Num_26', 'Den_26', 'Site_Name', 'Contract_ID', 'specialist_site_type_code']) # Dataframe template for result
    
    claimExists = False
    
    i = 0
    while i < len(df_26):
       
        js_id = df_26.Job_Seeker_ID.values[i]
        siteName = df_26.Site_Name.values[i]
        contractID = df_26.Contract_ID.values[i]
        specialistCode = df_26.specialist_site_type_code.values[i]
        num_26_full = df_26.Num_26_full.values[i] 
        den_26_full = df_26.Den_26_full.values[i]
        Den_26_full = 0
    
        Num_26_full = num_26_full
        
        # pathway
        num_26_path = df_26.Num_26_path.values[i] 
        den_26_path = df_26.Den_26_path.values[i]
        Den_26_path = 0
    
        Num_26_path = num_26_path
        
        #work assist/bonus
        num_26_wrkast = df_26.Num_26_wrkast.values[i] 
        den_26_wrkast = df_26.Den_26_wrkast.values[i]
        Den_26_wrkast = 0
    
        Num_26_wrkast = num_26_wrkast
        

        x = 0
        group = g.get_group(js_id)
        n = len(group)
        week_26 = pd.DateOffset(days = 182)
        stopLoop = False
        
        while x < n or not stopLoop:
            comDate = pd.to_datetime(group.COMMENCEMENT_DATE.values[0], format='%Y-%m-%d') # commencement date should be the first index 
            claim = group.Claim_Rate_Type.values[x]
            availDate = group.CLAIM_AVAILABLE_DATE.values[x]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[0]) # exit date should be recorded FOR ALL
            invalid_date = pd.to_datetime('2001-01-01', format='%Y-%m-%d')

          
            if den_26_full == maxDen or den_26_path == maxDen or den_26_wrkast == maxDen:
                pass # return -> denominated maxDen = 8 times, or maxNum/maxDen
            
            if (num_26_full == 0 and den_26_full == 0):
                if (num_26_full == 0 and den_26_full == 0):
                
                    Num_26_full = 0
                    Den_26_full = 0
                    stopLoop = True
                    break 
      
        
            elif comDate <= invalid_date:
                Num_26_full = 0
                Den_26_full += 0
                Num_26_path = 0
                Den_26_path += 0
                
            else: 
               
                # looping through the outcomes
                if (group.PLACEMENT_STATUS_CODE.values[x] == 'Exited') and (group.SUSPENSION_DAYS.values[x] > 0): # current entry is showing Exited -> check if exitDate is  
                    period_26 = comDate + week_26 
                    if (group.SUSPENSION_FROM_DATE.values[x] >= comDate) and (group.SUSPENSION_ACTUAL_END_DATE.values[x] <= period_26): 
                        # then suspension is applicable to this outcome and period
                        period_26 = comDate + week_26 + pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[x]))
                       
                        # check if participant has exited before the 26 week outcome was due: 
                        if exitDate < period_26: 
                            # then don't include at all -> wipe all numerator/denominator
                            Num_26_full = 0 
                            Den_26_full += 0
                            Num_26_path = 0
                            Den_26_path += 0
                            pass
                    
                    

                # loop through the quarters and compare with claim_available_date -> check which quarter it occurs in
                else:
                    
                    if (claim == 'DES 26wk Full Outcome') and (group.Claim_Status.values[x] == 'Approved'): # if 26 week outcome claim exists - validate the number for the denominator -> reset
                        Den_26_full = 1 
                        Num_26_full = 1
                        pass
                    
                    elif (claim == 'DES 26wk Pathway Outcome') and (group.Claim_Status.values[x] == 'Approved'): # if 26 week outcome claim exists - validate the number for the denominator -> reset
                        Den_26_path = 1 
                        Num_26_path = 1
                        pass
                        
                    elif claim == 'DES 52wk Full Outcome' or claim == 'DES 13wk Full Outcome':
                        Num_26_full += 0 
                        Den_26_full += 0 
                        pass
                    # 26 week outcome may already be recorded -> or may be in later entries
                    
                    else:
                        period_26 = comDate + week_26
                    # check if a 26 week full outcome exists in ANY of the group values
                        
                    z = 0
                    s = 0
                    e = 1
                    while z < len(qtrs) and not stopLoop:
                        start = pd.to_datetime(qtrs[z][s])
                        end = pd.to_datetime(qtrs[z][1])
                        
                        if z == len(qtrs) - 1:  # This is the last iteration
                            stopLoop = True
                        if (Den_26_full == maxDen or Num_26_full == maxNum) or (Den_26_path == maxDen or Num_26_path == maxNum) or (Den_26_wrkast == maxDen or Num_26_wrkast == maxNum):
                            stopLoop = True
                            break
                        else:
                            if 'DES 26wk Full Outcome' in group.Claim_Rate_Type.values:
                                # if the iteration -> loop is at the beginning of the current group -> get the commencement date of the FIRST row for the 26 week full outcome: 
                                outcomeInd = (group.Claim_Rate_Type.values == 'DES 26wk Full Outcome').argmax()
                                comDate = group.COMMENCEMENT_DATE.values[outcomeInd]
                                # get EXIT date for this here
                        
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[outcomeInd], format='%Y-%m-%d' ) # exit date should be recorded FOR ALL
                                ## TO-DO - investigate whether a SUSPENSION period occurs here and if exitDate <= period_26 (comDate + suspension_period + 26 weeks)
                            elif ('DES 26wk Full Outcome' not in group.Claim_Rate_Type.values ) and ('DES 26wk Pathway Outcome' in group.Claim_Rate_Type.values):    # all values are 26 week PATHWAY outcomes
                                pathwayInd = (group.Claim_Rate_Type.values == 'DES 26wk Full Outcome').argmax()
                                comDate = group.COMMENCEMENT_DATE.values[pathwayInd]
                    
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[pathwayInd], format='%Y-%m-%d' ) # exit date should be recorded FOR ALL
                            # check here how many times the participant denominates ->
                            if (comDate >= start) and (comDate <= end) and (group.PLACEMENT_STATUS_CODE.values[x] == 'Commenced'): # check if the commencement date is in the current quarter: 
                                
                                period_26 = comDate + week_26 # denominates if and only if at least 26 weeks has passed: 26 weeks + comDate -> falls into the current quarter, if not pass
                                if claim == 'DES 26wk Pathway Outcome':
                                    Den_26_path += 1
                                    
                            
                                elif (exitDate >= start) and (exitDate <= end) and (period_26 >= start) and (period_26 <= end): #exit date is in the current quarter -> then denominates and then exists
                                
                                    Den_26_full += 1 
                                    stopLoop = True
                                    break
                                elif (availDate >= start) and (availDate <= end) and (period_26 >= start) and (period_26 <= end): # in the current quarter -> exit date not counted
                                    Den_26_full += 1
                                    stopLoop = True
                                    break
                                   
                                #add here if exit before period_26
                                elif (exitDate < period_26):
                                
                                    Den_26_full = 0
                                    stopLoop = True
                                    break
                            


                            else: # commencement date has already passed, OR commencement date is later
                                # need to stop at participant EXIT_DATE - get exit date of the 26 week full outcome or pathway outcome:
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[0], format='%Y-%m-%d') # exit date should be recorded FOR ALL
                                period_26 =  comDate + week_26
                                
                                if (comDate > end): # commencement date is later ->
                                    pass
                                    
                                elif exitDate < period_26:
                     
                                    Den_26_full = 0
                                    stopLoop = True
                                    break 
                                
                                else: 
                               
                                    # check if exitDate is within the current quarter being traversed through
                                    if (exitDate >= start) and (exitDate <= end):
                                        if period_26 >= start and period_26 <= end: # duration at least 26 weeks OCCURS during this quarter and they exit in this quarter
                                            Den_26_full += 1
                                            stopLoop = True
                                            break
                                        elif exitDate < period_26:
                                            Den_26_full = 0
                                            stopLoop = True
                                            break
                                        else:
                                
                                            Den_26_full += 1
                                            stopLoop = True
                                            break
                                            
                                    elif (period_26 >= start and period_26 <= end) or (period_26 < start) and not (start <= exitDate <= end):
                                        
                                        Den_26_full += 1

                    
                                       
                                    else:
                                        Den_26_full += 0
                                       
                                
                        z += 1
                        
                   
                        
                    
                

                  

                x += 1
        

        if Den_26_full >= maxDen or Den_26_path >= maxDen:
            Den_26_path = 8  # set threshold
        data = pd.DataFrame({'Job_Seeker_ID' : [js_id], 'Num_26_full' : num_26_full, 'Den_26_full' : Den_26_full, 'Num_26_path' : num_26_path, 'Den_26_path' : Den_26_path, 'Num_26_wrkast' : num_26_wrkast, 'Den_26_wrkast' : Den_26_wrkast, 'Site_Name' : [siteName], 'Contract_ID' : [contractID], 'specialist_site_type_code' : [specialistCode]})
        df_post = df_post.append(data)

            

            
        
        i += 1
            





    return df_post # returns post-quarterisation dataframe 




In [529]:

x = agg_outcomes_26(df_pre, g, num_codes, den_codes, com_codes)

In [530]:

df_post_26 = post_quarter_26(x, df_pre, g, qtrs)

df_post_26_grouped = df_post_26.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code', 'Job_Seeker_ID']).agg({
    'Num_26_full': 'sum',
    'Den_26_full': 'sum',
    'Num_26_path': 'sum',
    'Den_26_path': 'sum',
    'Num_26_wrkast': 'sum',
    'Den_26_wrkast': 'sum'
})

df_post_26_grouped = df_post_26_grouped.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code']).agg({
    'Num_26_full': 'sum',
    'Den_26_full': 'sum',
    'Num_26_path': 'sum',
    'Den_26_path': 'sum',
    'Num_26_wrkast': 'sum',
    'Den_26_wrkast': 'sum'
})


In [531]:
df_post_26_grouped['Den_26_path'] = (df_post_26_grouped['Den_26_full'] - df_post_26_grouped['Num_26_full']) + df_post_26_grouped['Num_26_path']


In [532]:
output_str = 'C:/Users/AManalo/star_ratings_new/prediction_model/' + period + '/26_week_outcomes_sites.csv'
df_post_26_grouped
df_post_26_grouped.to_csv(output_str)

In [533]:
qtrs

[(numpy.datetime64('2019-10-01T00:00:00.000000000'),
  numpy.datetime64('2019-12-31T00:00:00.000000000')),
 (numpy.datetime64('2020-01-01T00:00:00.000000000'),
  numpy.datetime64('2020-03-31T00:00:00.000000000')),
 (numpy.datetime64('2020-04-01T00:00:00.000000000'),
  numpy.datetime64('2020-06-30T00:00:00.000000000')),
 (numpy.datetime64('2020-07-01T00:00:00.000000000'),
  numpy.datetime64('2020-09-30T00:00:00.000000000')),
 (numpy.datetime64('2020-10-01T00:00:00.000000000'),
  numpy.datetime64('2020-12-31T00:00:00.000000000')),
 (numpy.datetime64('2021-01-01T00:00:00.000000000'),
  numpy.datetime64('2021-03-31T00:00:00.000000000')),
 (numpy.datetime64('2021-04-01T00:00:00.000000000'),
  numpy.datetime64('2021-06-30T00:00:00.000000000')),
 (numpy.datetime64('2021-07-01T00:00:00.000000000'),
  numpy.datetime64('2021-09-30T00:00:00.000000000'))]

In [534]:
output_str = 'C:/Users/AManalo/star_ratings_new/prediction_model/' + period + '/df_post_26.csv'
df_post_26.to_csv(output_str)

### Separating the performance measures 26 outcomes 

## 52 week outcomes

### Pre-quarterisation function (52 week outcome)

In [535]:

def get_52_outcome(jobSeekerEntry, num_codes, den_codes, com_codes): # takes a row (groupBy on Job_Seeker_ID, list of eligible numerator phase codes + denominator phase codes
    # sort jobSeekerEntry by COMMENCEMENT_DATE
    endBounds = endDate
    startBounds = startDate
    jobSeekerEntry = jobSeekerEntry.sort_values(by=['COMMENCEMENT_DATE'])
    group = jobSeekerEntry # group containing the attributes for one job_seeker_ID
    id = group.index.values.astype(int)[0]
    # initialise numerator and denominator 
    num_52_full = 0
    den_52_full = 0
    num_52_path = 0 
    den_52_path = 0
    df_52 = pd.DataFrame()
    # TEMP ----------- REMOVE
    df_52['Site_Name'] = [group.Site_Name.values[0]]
    df_52['Contract_ID'] = [group.Contract_ID.values[0]]
    df_52['specialist_site_type_code'] = [group.specialist_site_type_code.values[0]]

    # TEMP -------------------
    # place id into dataframe 
    df_52['Job_Seeker_ID'] = [group.Job_Seeker_ID.values[0]]
    # initialise the variables in df_52
    df_52['Num_52_full'] = [num_52_full]
    df_52['Den_52_full'] = 0
    df_52['Num_52_path'] = [num_52_path]
    df_52['Den_52_path'] = 0

   
    week_52 = pd.DateOffset(days = 364) # 52 weeks -> 26 + 26 = 52 weeks -> days
    eduStr = ['DES 52wk Full Outcome Education']
    # Check number of entries in group
    targetStr = ['DES 52wk Full Outcome', 'DES 52wk Pathway Outcome'] #valid outcomes
    num_entries = int(len(df_entry))
    noClaim = 'na'
    valid_claims = ['Approved', 'Pending', 'Lodged'] 
    exitDate = group.EXIT_DATE.values[0] # if exit date exists ->  exit date is the same for all entries

# now loop ->
    j = 0
    comDate =  group.COMMENCEMENT_DATE.values[0]
    exitDate = group.EXIT_DATE.values[0] 
    while j < len(group.COMMENCEMENT_DATE.values):
        # commencement check ->
        if group.PLACEMENT_STATUS_CODE.values[j] == 'Commenced':
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_52 = comDate + week_52
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension
                # check suspension days is in the 2 year assessment period:
                 if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_52): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_52 = comDate + week_52 + sus_days
            else:

                pass

            # claim rate types:
            if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                # check if participant exits before claim_available_date ->
            
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if (exitDate < period_52):
                    if comDate > exitDate: # if commenced again after exit date:
                        # get the claim_rate_type -> 
                        if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                            num_52_full = 1
                            den_52_full = 1
                            df_52['Num_52_full'] = [num_52_full]
                            df_52['Den_52_full'] = 1
                            return df_52

                        elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                            num_52_path = 1
                            den_52_path = 1
                            df_52['Num_52_path'] = [num_52_path]
                            df_52['Den_52_path'] = 1
                            return df_52

                    else:
                        num_52_full = 0
                        den_52_full = 0
                        num_52_path = 0 
                        den_52_path = 0


                else: # check claim type 
                    if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                        num_52_full = 1
                        den_52_full = 1
                        df_52['Num_52_full'] = [num_52_full]
                        df_52['Den_52_full'] = 1
                        return df_52

                    elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                        num_52_path = 1
                        den_52_path = 1
                        df_52['Num_52_path'] = [num_52_path]
                        df_52['Den_52_path'] = 1
                        return df_52
            elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_52 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if exitDate < period_52: # duration in assistance is less than 52 weeks
                    if comDate > exitDate: # if commenced again after exit date:
                        num_52_full += 0
                        den_52_full = 1


                    else:
                        num_52_full = 0
                        den_52_full = 0
                        num_52_path = 0 
                        den_52_path = 0

                   
                else: # check duration in assistance is at least period_52
                    num_52_full += 0
                    den_52_full = 1 #education outcomes are counted in the denominator of the full outcome

            elif group.Claim_Rate_Type.values[j] == noClaim: 
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if exitDate < period_52: # duration in assistance is less than 52 weeks
                    num_52_full = 0
                    den_52_full = 0
                    num_52_path = 0 
                    den_52_path = 0

                else: # just denominate in the full outcome
                    num_52_full += 0 
                    den_52_full = 1
            else: # either 14 or 52 week claim
                num_52_full = 0
                den_52_full = 0

        else: # has not commenced ->
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_52 = comDate + week_52
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension 
                # check suspension days is in the 2 year assessment period:
                if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_52): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_52 = comDate + week_52 + sus_days
                
            # check if code is SUS_EA: (Counted in denominator if completed more than 52 weeks with us. Includes suspension period)
            if group.PLACE_REF.values[j] == 'SUS_EA':
                if group.CLAIM_AVAILABLE_DATE.values[j] >= period_52:
                    if (group.Claim_Rate_Type.values[j] in targetStr) and (group.Claim_Status.values[j] in valid_claims):
                       
                        if (period_52 <= startBounds) or (period_52 >= endBounds):
                                num_52_full = 0 
                                den_52_full = 0 # out of bounds
                        else:
                            # Check the claim type ->
                            if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                                num_52_full = 1
                                den_52_full = 1
                                df_52['Num_52_full'] = [num_52_full]
                                df_52['Den_52_full'] = 1
                                return df_52

                            elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                                num_52_path = 1
                                den_52_path = 1
                                df_52['Num_52_path'] = [num_52_path]
                                df_52['Den_52_path'] = 1
                                return df_52
                    else:
                        if (period_52 <= startBounds) or (period_52 >= endBounds):
                            num_52_full = 0 
                            den_52_full = 0 # out of bounds
                        else:
                            if (group.Claim_Rate_Type.values[j] == 'DES 52wk Pathway Outcome'): #this outcome IS NOt in the denominator
                                den_52_full += 0 
                                den_52_path = 1
                            else: # check the claim types and denominate these
                                if group.Claim_Rate_Type.values[j] == targetStr[0]: # full outcome
                                    den_52_full = 1

                                else: # NA claim
                                    den_52_full = 1

                else: # no claim_availble date or claim available date is less than period_52
                    if group.CLAIM_AVAILABLE_DATE.values[j] < period_52 and group.Claim_Rate_Type.values[j] == 'na':
                        # check duration in assistance
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate > period_52: # exits after 52 weeks
                            num_52_full += 0
                            den_52_full = 1
                        else: # exits before 52 weeks
                            num_52_full = 0
                            den_52_full = 0
                    else: # could be suspended but still record a claim ->
                        comDate = group.COMMENCEMENT_DATE.values[j]
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        period_52 = comDate + week_52
                        # check suspensions
                        if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension
                            # check suspension days is in the 2 year assessment period:
                            if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_52): # suspension in valid period ->
                                sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                                period_52 = comDate + week_52 + sus_days
                        else:
                            pass
 
                       
                        # claim_available_date could be valid and occur before period_52:
                        # claim rate types:
                        if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                            # check if participant exits before claim_available_date ->
                        
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if (exitDate < period_52):
                                # check if commences AFTER exit again:

                                if comDate > exitDate:
                                    if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                                        num_52_full = 1
                                        den_52_full = 1
                                        df_52['Num_52_full'] = [num_52_full]
                                        df_52['Den_52_full'] = 1
                                        return df_52

                                    elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                                        num_52_path = 1
                                        den_52_path = 1
                                        df_52['Num_52_path'] = [num_52_path]
                                        df_52['Den_52_path'] = 1
                                        return df_52 



                                else:
                                    num_52_full = 0
                                    den_52_full = 0
                                    num_52_path = 0 
                                    den_52_path = 0


                            else: # check claim type 
                                if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                                    num_52_full = 1
                                    den_52_full = 1
                                    df_52['Num_52_full'] = [num_52_full]
                                    df_52['Den_52_full'] = 1
                                    return df_52

                                elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                                    num_52_path = 1
                                    den_52_path = 1
                                    df_52['Num_52_path'] = [num_52_path]
                                    df_52['Den_52_path'] = 1
                                    return df_52
                        elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_52 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if exitDate < period_52: # duration in assistance is less than 52 weeks
                                if comDate > exitDate: # commences again after exiting
                                    num_52_full += 0
                                    den_52_full = 1
                                else:
                                    num_52_full = 0
                                    den_52_full = 0
                                    num_52_path = 0 
                                    den_52_path = 0

                                
                            else: # check duration in assistance is at least period_52
                                num_52_full += 0
                                den_52_full = 1 #education outcomes are counted in the denominator of the full outcome

                        elif group.Claim_Rate_Type.values[j] == noClaim: 
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if exitDate < period_52: # duration in assistance is less than 52 weeks
                                num_52_full = 0
                                den_52_full = 0
                                num_52_path = 0 
                                den_52_path = 0

                            else: # just denominate in the full outcome
                                num_52_full += 0 
                                den_52_full = 1
                        else: # either 14 or 52 week claim
                            num_52_full = 0
                            den_52_full = 0
                
                                        
            else: # TRN or EXT
                ext_tfn = ['Exited', 'Transferred']
                if group.PLACEMENT_STATUS_CODE.values[j] in ext_tfn:
                    exitDate = pd.to_datetime(group.EXIT_DATE.values[j])

                    if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                        # check if participant exits before claim_available_date ->
                    
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if (exitDate < period_52):
                            if comDate > exitDate: # commenced after exit
                                if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                                    num_52_full = 1
                                    den_52_full = 1
                                    df_52['Num_52_full'] = [num_52_full]
                                    df_52['Den_52_full'] = 1
                                    return df_52

                                elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                                    num_52_path = 1
                                    den_52_path = 1
                                    df_52['Num_52_path'] = [num_52_path]
                                    df_52['Den_52_path'] = 1
                                    return df_52
                                
                            
                            else:
                                num_52_full = 0
                                den_52_full = 0
                                num_52_path = 0 
                                den_52_path = 0


                        else: # check claim type 
                            if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                                num_52_full = 1
                                den_52_full = 1
                                df_52['Num_52_full'] = [num_52_full]
                                df_52['Den_52_full'] = 1
                                return df_52

                            elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                                num_52_path = 1
                                den_52_path = 1
                                df_52['Num_52_path'] = [num_52_path]
                                df_52['Den_52_path'] = 1
                                return df_52
                    elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_52 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate < period_52: # duration in assistance is less than 52 weeks
                            if comDate > exitDate: 
                                num_52_full += 0
                                den_52_full = 1
                            else:
                                num_52_full = 0
                                den_52_full = 0
                                num_52_path = 0 
                                den_52_path = 0


                        else: # check duration in assistance is at least period_52
                            num_52_full += 0
                            den_52_full = 1 #education outcomes are counted in the denominator of the full outcome

                    elif group.Claim_Rate_Type.values[j] == noClaim: 
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate < period_52: # duration in assistance is less than 52 weeks
                            num_52_full = 0
                            den_52_full = 0
                            num_52_path = 0 
                            den_52_path = 0

                        else: # just denominate in the full outcome
                            num_52_full += 0 
                            den_52_full = 1
                    else: # either 14 or 52 week claim
                        num_52_full = 0
                        den_52_full = 0
                
        j += 1

    df_52['Num_52_full'] = [num_52_full]
    df_52['Den_52_full'] = den_52_full
    df_52['Num_52_path'] = [num_52_path]        
    df_52['Den_52_path'] = den_52_path

    return df_52

df_entry = g.get_group(451780)

f = get_52_outcome(df_entry, num_codes, den_codes, com_codes) # pass results to a new dataframe and append each time
f






,Site_Name,Contract_ID,specialist_site_type_code,Job_Seeker_ID,Num_52_full,Den_52_full,Num_52_path,Den_52_path
0,AimBig Employment CALOUNDRA,0212875D,MUSK,451780,0,0,0,0


# aggregation function for pre-quarter function:

In [536]:
# adds outcome to df_52
def agg_outcomes_52(df_pre, g, num_codes, den_codes, com_codes):
    df_52 = pd.DataFrame(columns= ['Job_Seeker_ID', 'Num_52', 'Den_52', 'Site_Name', 'Contract_ID', 'specialist_site_type_code']) # empty
    # takes grouped by job_seeker_id dataframe, passes each row to get_no#_outcome



    ids = df_pre["Job_Seeker_ID"].unique().astype("int64")
    for id in df_pre['Job_Seeker_ID'].unique():
        if math.isnan(id):
            pass
        else:
            df_entry = g.get_group(id)
            f = get_52_outcome(df_entry, num_codes, den_codes, com_codes)
            df_52 = df_52.append(f)
            

    return df_52 # returns output from get_52_outcome 

### Pre-quarterisation 52 output file:

In [537]:
### Testing the aggregate function for 13 outcomes
df_52 = agg_outcomes_52(df_pre, g, num_codes, den_codes, com_codes)


warnings.filterwarnings('ignore')

print('Number of IDs for 52 week outcomes: ' + str(len(df_52)))

output_str = 'C:/Users/AManalo/star_ratings_new/prediction_model/' + period + '/df_52_new.csv'
df_52.to_csv(output_str)

Number of IDs for 52 week outcomes: 3363


In [538]:
df_52

,Job_Seeker_ID,Num_52,Den_52,Site_Name,Contract_ID,specialist_site_type_code,Num_52_full,Den_52_full,Num_52_path,Den_52_path
0,9386301003,NaN,NaN,AimBig Employment WOY WOY,0212812K,NaN,0.0,0.0,0.0,0.0
0,8653691003,NaN,NaN,AimBig Employment FAIRFIELD,0212899A,AALL,0.0,1.0,0.0,0.0
0,7870290004,NaN,NaN,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,1.0,0.0,0.0
0,6498977509,NaN,NaN,AimBig Employment LEICHHARDT,0212845J,AALL,0.0,0.0,0.0,0.0
0,6638331004,NaN,NaN,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
0,7495948709,NaN,NaN,AimBig Employment MOUNT DRUITT,0212853J,MENH,0.0,1.0,0.0,0.0
0,5553772007,NaN,NaN,AimBig Employment INALA,0212796E,NaN,0.0,0.0,0.0,0.0
0,6688566909,NaN,NaN,AimBig Employment Pty Ltd ST ALBANS,0212822B,MENH,0.0,0.0,0.0,0.0
0,13302780,NaN,NaN,AimBig Employment FAIRFIELD,0212844H,NaN,0.0,0.0,0.0,0.0


## Post-Quarterisation 52 week outcome

In [539]:
endBounds = endDate

startBounds = startDate

def post_quarter_52(df_52, df_pre, g, qtrs):
    '''Post quarterisation performance for 52 week full outcomes. Receives df_pre as an input, and outputs the post_quarterised denominators for each eligible participant.
    df_52 is used to check if a participant has numerated, df_pre is used to feed in the job_seeker_id to 'g' - the groupBy object and check whether the job seeker denominates - if no numerator in the df_52 exists.'''
    # maximum amount of times they numerate/denominate:
    maxDen = 8
    maxNum = 1

    df_post = pd.DataFrame(columns=['Job_Seeker_ID', 'Num_52', 'Den_52', 'Site_Name', 'Contract_ID', 'specialist_site_type_code']) # Dataframe template for result
    
    claimExists = False
    
    i = 0
    while i < len(df_52):
       
        js_id = df_52.Job_Seeker_ID.values[i]
        siteName = df_52.Site_Name.values[i]
        contractID = df_52.Contract_ID.values[i]
        specialistCode = df_52.specialist_site_type_code.values[i]
        num_52_full = df_52.Num_52_full.values[i] 
        den_52_full = df_52.Den_52_full.values[i]
        Den_52_full = 0
    
        Num_52_full = num_52_full
        
        # pathway
        num_52_path = df_52.Num_52_path.values[i] 
        den_52_path = df_52.Den_52_path.values[i]
        Den_52_path = 0
    
        Num_52_path = num_52_path
   

        x = 0
        group = g.get_group(js_id)
        n = len(group)
        week_52 = pd.DateOffset(days = 182)
        stopLoop = False
        
        while x < n or not stopLoop:
            comDate = pd.to_datetime(group.COMMENCEMENT_DATE.values[0], format='%Y-%m-%d') # commencement date should be the first index 
            claim = group.Claim_Rate_Type.values[x]
            availDate = group.CLAIM_AVAILABLE_DATE.values[x]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[0]) # exit date should be recorded FOR ALL
            invalid_date = pd.to_datetime('2001-01-01', format='%Y-%m-%d')

          
            if den_52_full == maxDen or den_52_path == maxDen:
                pass # return -> denominated maxDen = 8 times, or maxNum/maxDen
            
            if (num_52_full == 0 and den_52_full == 0):
                if (num_52_full == 0 and den_52_full == 0):
                
                    Num_52_full = 0
                    Den_52_full = 0
                    stopLoop = True
                    break 
      
        
            elif comDate <= invalid_date:
                Num_52_full = 0
                Den_52_full += 0
                Num_52_path = 0
                Den_52_path += 0
                
            else: 
               
                # looping through the outcomes
                if (group.PLACEMENT_STATUS_CODE.values[x] == 'Exited') and (group.SUSPENSION_DAYS.values[x] > 0): # current entry is showing Exited -> check if exitDate is  
                    period_52 = comDate + week_52 
                    if (group.SUSPENSION_FROM_DATE.values[x] >= comDate) and (group.SUSPENSION_ACTUAL_END_DATE.values[x] <= period_52): 
                        # then suspension is applicable to this outcome and period
                        period_52 = comDate + week_52 + pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[x]))
                       
                        # check if participant has exited before the 52 week outcome was due: 
                        if exitDate < period_52: 
                            # then don't include at all -> wipe all numerator/denominator
                            Num_52_full = 0 
                            Den_52_full += 0
                            Num_52_path = 0
                            Den_52_path += 0
                            pass
                    
                    

                # loop through the quarters and compare with claim_available_date -> check which quarter it occurs in
                else:
                    
                    if (claim == 'DES 52wk Full Outcome') and (group.Claim_Status.values[x] == 'Approved'): # if 52 week outcome claim exists - validate the number for the denominator -> reset
                        Den_52_full = 1 
                        Num_52_full = 1
                        pass
                    
                    elif (claim == 'DES 52wk Pathway Outcome') and (group.Claim_Status.values[x] == 'Approved'): # if 52 week outcome claim exists - validate the number for the denominator -> reset
                        Den_52_path = 1 
                        Num_52_path = 1
                        pass
                        
                    elif claim == 'DES 52wk Full Outcome' or claim == 'DES 13wk Full Outcome':
                        Num_52_full += 0 
                        Den_52_full += 0 
                        pass
                    # 52 week outcome may already be recorded -> or may be in later entries
                    
                    else:
                        period_52 = comDate + week_52
                    # check if a 52 week full outcome exists in ANY of the group values
                        
                    z = 0
                    s = 0
                    e = 1
                    while z < len(qtrs) and not stopLoop:
                        start = pd.to_datetime(qtrs[z][s])
                        end = pd.to_datetime(qtrs[z][1])
                        
                        if z == len(qtrs) - 1:  # This is the last iteration
                            stopLoop = True
                        if (Den_52_full == maxDen or Num_52_full == maxNum) or (Den_52_path == maxDen or Num_52_path == maxNum):
                            stopLoop = True
                            break
                        else:
                            if 'DES 52wk Full Outcome' in group.Claim_Rate_Type.values:
                                # if the iteration -> loop is at the beginning of the current group -> get the commencement date of the FIRST row for the 52 week full outcome: 
                                outcomeInd = (group.Claim_Rate_Type.values == 'DES 52wk Full Outcome').argmax()
                                comDate = group.COMMENCEMENT_DATE.values[outcomeInd]
                                # get EXIT date for this here
                        
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[outcomeInd], format='%Y-%m-%d' ) # exit date should be recorded FOR ALL
                                ## TO-DO - investigate whether a SUSPENSION period occurs here and if exitDate <= period_52 (comDate + suspension_period + 52 weeks)
                            elif ('DES 52wk Full Outcome' not in group.Claim_Rate_Type.values ) and ('DES 52wk Pathway Outcome' in group.Claim_Rate_Type.values):    # all values are 52 week PATHWAY outcomes
                                pathwayInd = (group.Claim_Rate_Type.values == 'DES 52wk Full Outcome').argmax()
                                comDate = group.COMMENCEMENT_DATE.values[pathwayInd]
                    
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[pathwayInd], format='%Y-%m-%d' ) # exit date should be recorded FOR ALL
                            # check here how many times the participant denominates ->
                            if (comDate >= start) and (comDate <= end) and (group.PLACEMENT_STATUS_CODE.values[x] == 'Commenced'): # check if the commencement date is in the current quarter: 
                                
                                period_52 = comDate + week_52 # denominates if and only if at least 52 weeks has passed: 52 weeks + comDate -> falls into the current quarter, if not pass
                                if claim == 'DES 52wk Pathway Outcome':
                                    Den_52_path += 1
                                    
                            
                                elif (exitDate >= start) and (exitDate <= end) and (period_52 >= start) and (period_52 <= end): #exit date is in the current quarter -> then denominates and then exists
                                
                                    Den_52_full += 1 
                                    stopLoop = True
                                    break
                                elif (availDate >= start) and (availDate <= end) and (period_52 >= start) and (period_52 <= end): # in the current quarter -> exit date not counted
                                    Den_52_full += 1
                                    stopLoop = True
                                    break
                                   
                                #add here if exit before period_52
                                elif (exitDate < period_52):
                                
                                    Den_52_full = 0
                                    stopLoop = True
                                    break
                            


                            else: # commencement date has already passed, OR commencement date is later
                                # need to stop at participant EXIT_DATE - get exit date of the 52 week full outcome or pathway outcome:
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[0], format='%Y-%m-%d') # exit date should be recorded FOR ALL
                                period_52 =  comDate + week_52
                                
                                if (comDate > end): # commencement date is later ->
                                    pass
                                    
                                elif exitDate < period_52:
                     
                                    Den_52_full = 0
                                    stopLoop = True
                                    break 
                                
                                else: 
                               
                                    # check if exitDate is within the current quarter being traversed through
                                    if (exitDate >= start) and (exitDate <= end):
                                        if period_52 >= start and period_52 <= end: # duration at least 52 weeks OCCURS during this quarter and they exit in this quarter
                                            Den_52_full += 1
                                            stopLoop = True
                                            break
                                        elif exitDate < period_52:
                                            Den_52_full = 0
                                            stopLoop = True
                                            break
                                        else:
                                
                                            Den_52_full += 1
                                            stopLoop = True
                                            break
                                            
                                    elif (period_52 >= start and period_52 <= end) or (period_52 < start) and not (start <= exitDate <= end):
                                        
                                        Den_52_full += 1

                    
                                       
                                    else:
                                        Den_52_full += 0
                                       
                                
                        z += 1
                        
                   
                        
                    
                

                  

                x += 1
        

        if Den_52_full >= maxDen or Den_52_path >= maxDen:
            Den_52_path = 8  # set threshold
        data = pd.DataFrame({'Job_Seeker_ID' : [js_id], 'Num_52_full' : num_52_full, 'Den_52_full' : Den_52_full, 'Num_52_path' : num_52_path, 'Den_52_path' : Den_52_path, 'Site_Name' : [siteName], 'Contract_ID' : [contractID], 'specialist_site_type_code' : [specialistCode]})
        df_post = df_post.append(data)

            

            
        
        i += 1
            





    return df_post # returns post-quarterisation dataframe 




### Running the post_quarter_52 week function

In [540]:
df_post_52 = post_quarter_52(df_52, df_pre, g, qtrs)

output_str = 'C:/Users/AManalo/star_ratings_new/prediction_model/' + period + '/df_post_52.csv'

df_post_52.to_csv(output_str)

In [541]:
# output file
df_post_52

,Job_Seeker_ID,Num_52,Den_52,Site_Name,Contract_ID,specialist_site_type_code,Num_52_full,Den_52_full,Num_52_path,Den_52_path
0,9386301003,NaN,NaN,AimBig Employment WOY WOY,0212812K,NaN,0.0,0.0,0.0,0.0
0,8653691003,NaN,NaN,AimBig Employment FAIRFIELD,0212899A,AALL,0.0,8.0,0.0,8.0
0,7870290004,NaN,NaN,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,4.0,0.0,0.0
0,6498977509,NaN,NaN,AimBig Employment LEICHHARDT,0212845J,AALL,0.0,0.0,0.0,0.0
0,6638331004,NaN,NaN,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
0,7495948709,NaN,NaN,AimBig Employment MOUNT DRUITT,0212853J,MENH,0.0,0.0,0.0,0.0
0,5553772007,NaN,NaN,AimBig Employment INALA,0212796E,NaN,0.0,0.0,0.0,0.0
0,6688566909,NaN,NaN,AimBig Employment Pty Ltd ST ALBANS,0212822B,MENH,0.0,0.0,0.0,0.0
0,13302780,NaN,NaN,AimBig Employment FAIRFIELD,0212844H,NaN,0.0,0.0,0.0,0.0


In [542]:
df_post_52 = df_post_52.drop(columns=['Job_Seeker_ID'])
df_post_52

,Num_52,Den_52,Site_Name,Contract_ID,specialist_site_type_code,Num_52_full,Den_52_full,Num_52_path,Den_52_path
0,NaN,NaN,AimBig Employment WOY WOY,0212812K,NaN,0.0,0.0,0.0,0.0
0,NaN,NaN,AimBig Employment FAIRFIELD,0212899A,AALL,0.0,8.0,0.0,8.0
0,NaN,NaN,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,4.0,0.0,0.0
0,NaN,NaN,AimBig Employment LEICHHARDT,0212845J,AALL,0.0,0.0,0.0,0.0
0,NaN,NaN,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
0,NaN,NaN,AimBig Employment MOUNT DRUITT,0212853J,MENH,0.0,0.0,0.0,0.0
0,NaN,NaN,AimBig Employment INALA,0212796E,NaN,0.0,0.0,0.0,0.0
0,NaN,NaN,AimBig Employment Pty Ltd ST ALBANS,0212822B,MENH,0.0,0.0,0.0,0.0
0,NaN,NaN,AimBig Employment FAIRFIELD,0212844H,NaN,0.0,0.0,0.0,0.0


In [543]:
df_post_52 = df_post_52.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code'])['Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].sum()
df_post_52['Den_52_path'] = (df_post_52['Den_52_full'] - df_post_52['Num_52_full']) + df_post_52['Num_52_path']

output_str = 'C:/Users/AManalo/star_ratings_new/prediction_model/' + period + '/52_week_outcomes_sites.csv'
df_post_52.to_csv(output_str)


#### Caseload/Missing
- Caseload - identify low job seekers (<20 caseload) -> insufficient load -> identify contracts
- Count case load -> overall and let depar know - done
- missing data -> gaps we need to fill - done
- Identify people that are not getting an outcome at all - done
- Find proportion of data that needs to be cleaned (%)  - done



1) Caseload File


In [544]:
# total caseload:

#naVal = df_cases.isna().sum() * 100 / len(df_cases)
#missValDf = pd.DataFrame({'column_name': df_cases.columns, 'percent_missing': naVal})

#print('total cases: ' + str(len(df_cases)))
#missValDf.sort_values(by=['percent_missing'], ascending=False).head(17)



2) Claims File

In [545]:

# naVal = df_claims.isna().sum() * 100 / len(df_claims)
# missValDf = pd.DataFrame({'column_name': df_claims.columns, 'percent_missing': naVal})

# print('total claims: ' + str(len(df_claims)))
# missValDf.sort_values(by=['percent_missing'], ascending=False).head(3)



### Participants with no outcome recorded for Dec Star Rating 2021 - Job_Seeker_ID

In [546]:
# # participants without an outcome - get the ones with an outcome from the 'find' variable
# outcome = df_pre["Job_Seeker_ID"].drop_duplicates()
# df_pre_temp = df_pre_temp["Job_Seeker_ID"].drop_duplicates()
# vals = df_pre_temp.isin(outcome)
# # list of participants (Job_Seeker_ID) without an outcome

# # getting the false values:
# no_outcome = (vals != True).index.values



### Proportion of Data that needs Cleaning
1. Listing distinct values for each attribute and inspecting manually

In [547]:
df_pre.columns

Index(['Job_Seeker_ID', 'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE',
       'PLACEMENT_TYPE', 'Program', 'Contract_ID', 'Site_Name', 'ESA_Code',
       'specialist_site_type_code', 'COMMENCEMENT_DATE', 'EXIT_DATE',
       'Claim_ID', 'Claim_Rate_Type', 'Claim_Status', 'CLAIM_START_DATE',
       'CLAIM_AVAILABLE_DATE', 'CLAIM_END_DATE', 'SUSPENSION_FROM_DATE',
       'SUSPENSION_ACTUAL_END_DATE', 'OUTCOME_TYPE', 'SUSPENSION_DAYS',
       'PLACE_REF'],
      dtype='object')

In [548]:
# place_seq_num = df_pre['PLACEMENT_SEQUENCE_NUMBER'].drop_duplicates().values
# print('place_seq_num: ', np.sort(place_seq_num))

# job_seeker_id = df_pre['Job_Seeker_ID'].drop_duplicates().values
# #print('job_seekers: ', job_seeker_id)

# place_stat_cd = df_pre['PLACEMENT_STATUS_CODE'].drop_duplicates().values
# print('place_stat_cd: ', place_stat_cd)

# ref_phase_cd = df_pre['REFERRAL_PHASE_CODE'].drop_duplicates().values
# print('ref phase code: ', ref_phase_cd)

# commence_dt = df_pre['COMMENCEMENT_DATE'].drop_duplicates().values
# #print('commence_date: ', commence_dt)

# claim_id = df_pre['Claim_ID'].drop_duplicates().values
# print('claim_id: ', claim_id)

# claim_rate = df_pre['Claim_Rate_Type'].drop_duplicates().values
# print('claim_rate: ', claim_rate)

# claim_stat = df_pre['Claim_Status'].drop_duplicates().values
# print('claim stat: ', claim_stat)

# claim_start = df_pre['CLAIM_START_DATE'].drop_duplicates().values
# #print('claim_start: ', claim_start)

# claim_avail = df_pre['CLAIM_AVAILABLE_DATE'].drop_duplicates().values
# #print('claim_avail: ', claim_avail)

# claim_end = df_pre['CLAIM_END_DATE'].drop_duplicates().values
# #print('claim_end: ', claim_end)

# suspen_start = df_pre['SUSPENSION_FROM_DATE'].drop_duplicates().values
# #print('suspen_start_date: ', suspen_start)

# suspen_end = df_pre['SUSPENSION_ACTUAL_END_DATE'].drop_duplicates().values
# #print('suspen end: ', suspen_end)

# outcome_type = df_pre['OUTCOME_TYPE'].drop_duplicates().values
# print('outcome_type: ', outcome_type)



- Missing values for placement_sequence_number: 13, 15, 16, 17, 18, 19
- Min: 1, Max: 20

- Incorrect spelling for claim_rate: 'Post Placement Special Payment - DES DMS Job Seeke' - 'Job Seeker?'

# Testing

In [549]:
import io, os, sys, types
import unittest
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

### Coffs Harbour Test for 26 week outcomes
Contract: 0212826F

In [550]:
jobSeekers = pd.read_csv('C:/Users/AManalo/star_ratings_new/jobseeker_test.csv')
# job seekers
jobSeekers = jobSeekers["Job_Seeker_ID"].unique()

len(jobSeekers)

27

In [551]:
class TestNotebook(unittest.TestCase):
    # 13 week test cases:                                                                      
    def test_edu_13(self):
        '''If DES 13wk Full Outcome Education = Claim_Rate_Type, then den=1, num=0'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome Education' in df_entry.Claim_Rate_Type.values:
                out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                num = out['Num_13'][0]
                den = out['Den_13'][0]

                assert (num == 0), "Education 13 outcome should NOT numerate."
                assert (den == 1), "Education 13 outcome MUST denominate."

    def test_valid_13(self):
        '''Claim status should be Approved, Lodged, Pending for it to numerate and claim_available_date AFTER or EQUAL TO commencement_date'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                ind_13 = (df_entry.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                if df_entry.Claim_Status.values[ind_13] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_13] >= df_entry.COMMENCEMENT_DATE.values[ind_13]):
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]

                    assert (num == 1), "Valid 13 full outcome, must numerate."
                    assert (den == 1), "Valid 13 full outcome, must denominate."

                else: # Claim Status is not valid
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]

                    assert (num == 0), "Invalid 13 full outcome, doesn't numerate."
                    assert (den == 1), "Invalid 13 full outcome, must denominate."



    def test_13_date(self):
        '''If 13 week claim available date is BEFORE commencement date -> then 0/0'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                ind_13 = (df_entry.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                if df_entry.CLAIM_AVAILABLE_DATE.values[ind_13] < df_entry.COMMENCEMENT_DATE.values[ind_13]:
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]

                    assert (num == 0), "Invalid Claim_Available_Date, doesn't numerate."
                    assert (den == 0), "Invalid Claim_Available_Date, doesn't denominate."

    def test_13_exit(self):
        '''If a participant exits before 13 week claim is due, then Num=0,Den=0'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                ind_13 = (df_entry.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                claim_avail_date = pd.to_datetime(df_entry.CLAIM_AVAILABLE_DATE.values[ind_13])
                exit_date = pd.to_datetime(df_entry.EXIT_DATE.values[ind_13])
                if exit_date < claim_avail_date:
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]
                    try: 
                        assert (num == 0), "Exits before claim, doesn't numerate."
                        assert (den == 0), "Exits before claim, doesn't denominate."
                    except AssertionError as e:
                        print(e)
                        print(df_entry.Job_Seeker_ID)
                    


    def test_13_path(self):
        '''If a 13 week pathway outcome is recorded, then Num=0, Den=1. Participant denominates.'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                pass
            else:
                if 'DES 13wk Pathway Outcome' in df_entry.Claim_Rate_Type.values:
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]

                    assert (num == 0), "13 wk pathway doesn't numerate."
                    assert (den == 1), "13 wk pathway must denominate."
    
    ###### 26 week outcome tests
    def test_26_edu(self):
        '''26 week Education outcomes are not included. Num=0, Den=1'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 26wk Full Outcome Education' in df_entry.Claim_Rate_Type.values:
                out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                num = out['Num_26_full'][0]
                den = out['Den_26_full'][0]
                assert (num == 0), "Education 26 outcome not valid. Doesn't numerate."
                assert (den == 1), "Education 26 outcome included in the denominator."

    def test_26_valid_path(self):
        '''26 week Pathway outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses.'''
                
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 26wk Pathway Outcome' in df_entry.Claim_Rate_Type.values:
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Pathway Outcome').argmax()
                if df_entry.Claim_Status.values[ind_26] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_26] >= df_entry.COMMENCEMENT_DATE.values[ind_26]):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 1), "Valid 26 pathway, must numerate."
                    assert (den == 1), "Valid 26 pathway, must denominate."
                else: # Claim Status is not valid
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 0), "Invalid 26 pathway, doesn't numerate."
                    assert (den == 1), "Invalid 26 pathway, must denominate."

    def test_26_valid_full(self):
        '''26 week Full outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses.'''
                
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 26wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Full Outcome').argmax()
                if df_entry.Claim_Status.values[ind_26] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_26] >= df_entry.COMMENCEMENT_DATE.values[ind_26]):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 1), "Valid 26 outcome, must numerate." 
                    assert (den == 1), "Valid 26 outcome, must denominate."
                else: # Claim Status is not valid
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 0), "Invalid claim status, doesn't numerate."
                    assert (den == 1), "Invalid claim status, must denominate."
            

    def test_26_valid_bonus(self):
        '''26 week Bonus or 26 week work assist outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses'''
                
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 26wk Bonus' in df_entry.Claim_Rate_Type.values:
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Bonus').argmax()
                if df_entry.Claim_Status.values[ind_26] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_26] >= df_entry.COMMENCEMENT_DATE.values[ind_26]):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_wrkast'][0]
                    den = out['Den_26_wrkast'][0]
                    assert (num == 1), "Valid bonus claim, must numerate." 
                    assert (den == 1), "Valid bonus claim, must denominate."

                else: # Claim Status is not valid
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 0), "Invalid claim status, should not numerate."
                    assert (den == 1), "Invalid claim status, must denominate."
                
            elif 'DES 26wk Work Assist Outcome' in df_entry.Claim_Rate_Type.values:
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Work Assist Outcome').argmax()
                if df_entry.Claim_Status.values[ind_26] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_26] >= df_entry.COMMENCEMENT_DATE.values[ind_26]):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_wrkast'][0]
                    den = out['Den_26)_wrkast'][0]

                    assert (num == 1), "Valid 26wk Work Assist must numerate."
                    assert (den == 1), "Valid 26wk Work Assist must denominate."
                else: # Claim Status is not valid
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_wrkast'][0]
                    den = out['Den_26_wrkast'][0]

                    assert (num == 0), "Invalid 26 work assist claim status, no outcome."
                    assert (den == 1), "Invalid 26 work assist claim status, must denominate."

    def test_26_exit(self):
        '''If a participant exits before 26 week claim is due, then num_26 = 0, den_26 = 0'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if ('DES 26wk Full Outcome' in df_entry.Claim_Rate_Type.values):
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Full Outcome').argmax()
                exit_date = pd.to_datetime(df_entry.EXIT_DATE.values[ind_26])
                claim_avail_date = pd.to_datetime(df_entry.CLAIM_AVAILABLE_DATE.values[ind_26])
                if (exit_date) < (claim_avail_date):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]

                    assert (num == 0), "Exits before 26 full claim, doesn't numerate."
                    assert (den == 0), "Exits before 26 full claim, doesn't denominate."
            
            elif ('DES 26wk Pathway Outcome' in df_entry.Claim_Rate_Type.values):
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Pathway Outcome').argmax()
                exit_date = pd.to_datetime(df_entry.EXIT_DATE.values[ind_26])
                claim_avail_date = pd.to_date(df_entry.CLAIM_AVAILABLE_DATE.values[ind_26])
                if (exit_date) < (claim_avail_date):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_path'][0]
                    den = out['Den_26_path'][0]
          
                    assert (num == 0), "Exits before 26 pathway claim, doesn't numerate."
                    assert (den == 0), "Exits before 26 pathway claim, doesn't denominate."

if __name__ == '__main__':
    main = TestNotebook()
    # execute unit test
    import sys

    suite = unittest.TestLoader().loadTestsFromTestCase(TestNotebook)
    unittest.TextTestRunner(verbosity=4, stream=sys.stderr).run(suite)

test_13_date (__main__.TestNotebook)
If 13 week claim available date is BEFORE commencement date -> then 0/0 ... ERROR
test_13_exit (__main__.TestNotebook)
If a participant exits before 13 week claim is due, then Num=0,Den=0 ... ERROR
test_13_path (__main__.TestNotebook)
If a 13 week pathway outcome is recorded, then Num=0, Den=1. Participant denominates. ... ERROR
test_26_edu (__main__.TestNotebook)
26 week Education outcomes are not included. Num=0, Den=1 ... ERROR
test_26_exit (__main__.TestNotebook)
If a participant exits before 26 week claim is due, then num_26 = 0, den_26 = 0 ... ERROR
test_26_valid_bonus (__main__.TestNotebook)
26 week Bonus or 26 week work assist outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses ... ERROR
test_26_valid_full (__main__.TestNotebook)
26 week Full outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses. ... ERROR
test_26_valid_path (__main__.TestNotebook)
26 week Pathway outcomes are included. Num=1, Den=